# Tanzania Tourism Prediction - Prognozy dotyczące turystyki w Tanzanii
<h2>Autorzy:</h2><br>
<ul>
<li>Piotr Janiszek 247678</li>
<li>Kacper Białek 247629</li>
<li>Franciszek Pawlak 247756</li>
<li>Michał Korblit 242427</li>
</ul>

In [1]:
is_test_iteration = False

import warnings
warnings.filterwarnings("ignore")


<h3>Imports</h3>

In [2]:
import numpy as np
import optuna
import pandas as pd
import shap

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW
import optuna

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
import lightgbm as lgb

from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Batch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, RobustScaler
import optuna
import time
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.linear_model import HuberRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb


ModuleNotFoundError: No module named 'optuna'

<h3>Data loadind</h3>


In [3]:
df_train = pd.read_csv('data/Train.csv')
df_regions = pd.read_csv('data/regions.csv')
df_distance = pd.read_csv('data/distance_to_TZ.csv')
df_regions = df_regions.merge(df_distance, on='alpha-2', how='left')

df_gdp = pd.read_csv('data/gdp-per-capita-worldbank.csv')
df_gdp = df_gdp.rename(columns={'Code': 'alpha-3', 'GDP per capita, PPP (constant 2021 international $)': 'gdp_per_capita_usd'})
df_gdp = df_gdp.drop(columns=['time', 'Entity', 'Year'])
df_regions = df_regions.merge(df_gdp, on='alpha-3', how='left')

corrections = {
    'SWIZERLAND': 'SWITZERLAND',
    'MALT': 'MALTA',
    'BURGARIA': 'BULGARIA',
    'DRC': 'CONGO (DEMOCRATIC REPUBLIC OF THE)',
    'KOREA': 'SOUTH KOREA',
    'SWAZILAND': 'ESWATINI',
    'UKRAIN': 'UKRAINE',
    'TRINIDAD TOBACCO': 'TRINIDAD AND TOBAGO',
    'COMORO': 'COMOROS',
    'COSTARICA': 'COSTA RICA',
    'PHILIPINES': 'PHILIPPINES',
    'IVORY COAST': "CÔTE D'IVOIRE",
    'DJIBOUT': 'DJIBOUTI',
    'MORROCO': 'MOROCCO',
    'UNITED STATES OF AMERICA': 'UNITED STATES',
    'UAE': 'UNITED ARAB EMIRATES',
    'SCOTLAND': 'UNITED KINGDOM',
    'CAPE VERDE': 'CABO VERDE',
}

df_train['country'] = df_train['country'].replace(corrections)
df_regions = df_regions[['name', 'sub-region', 'distance_km', 'gdp_per_capita_usd']]
df_regions = df_regions.rename(columns={'sub-region': 'region'})
df_regions['name'] = df_regions['name'].str.upper()
df_train = pd.merge(df_train, df_regions, how='left', left_on='country', right_on='name')
df_train = df_train.drop(columns=['name'])

print(df_train.columns)
print(df_regions.columns)


NameError: name 'pd' is not defined

<h3>Data Cleaning</h3>

In [ ]:
count = df_train.isna().any(axis=1).sum()
df_train.loc[df_train['most_impressing'].isna(), 'most_impressing'] = 'No comments'
count2 = df_train.isna().any(axis=1).sum()

mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])
mean = df_train.loc[mask_valid, 'total_cost_per_person'].mean()
print("Średnia bez wierszy z zerową sumą:", mean)


# idzie przez kolumny z nan
for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    if (pd.isna(row['total_male']) | pd.isna(row['total_female'])) & (not (pd.isna(row['total_male']) & pd.isna(row['total_female']))):
        if pd.isna(row['total_male']):
            difference = row['total_cost'] - (row['total_female'] * mean)
            person_left = difference / mean
            person_left = round(person_left, 0)

            if person_left < 0:
                person_left = 0
            #print(f"Inserting {person_left}")
            df_train.loc[index, 'total_male'] = person_left

        else:
            difference = row['total_cost'] - (row['total_male'] * mean)

            person_left = difference / mean
            person_left = round(person_left, 0)

            if person_left < 0:
                person_left = 0
            #print(f"Inserting {person_left}")
            df_train.loc[index, 'total_female'] = person_left

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    # Completing the travel_with field with the value Alone, when the number of people shows that he/she travels alone
    if pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 1):
        df_train.loc[index, 'travel_with'] = 'Alone'


In [ ]:
# Random forest imputation
mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])
mean = df_train.loc[mask_valid, 'total_cost_per_person'].mean()

features = ['region', 'age_group', 'total_female',
       'total_male', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

# Określ dozwolone kategorie - Model 1
allowed_categories = ['Friends/Relatives', 'Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp = df_train[df_train['travel_with'].isin(allowed_categories)].copy()
X_train = pd.get_dummies(df_train_imp[features], columns=features_cat)
y_train = df_train_imp['travel_with']
# Trenowanie modelu
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Określ dozwolone kategorie - Model 2
allowed_categories2 = ['Children', 'Friends/Relatives', 'Spouse', 'Spouse and Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp2 = df_train[df_train['travel_with'].isin(allowed_categories2)].copy()
X_train2 = pd.get_dummies(df_train_imp2[features], columns=features_cat)
y_train2 = df_train_imp2['travel_with']
# Trenowanie modelu
rf2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf2.fit(X_train2, y_train2)

# Określ dozwolone kategorie - Model 3
features3 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat3 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

def designate_sex(row):
    if (row['travel_with'] == 'Alone') and ((row['total_male'] + row['total_female']) == 1):
        # Jeśli total_male == 1, przyjmujemy, że to mężczyzna, w przeciwnym razie kobieta
        return 'Male' if row['total_male'] == 1 else 'Female'
    else:
        return None

df_train['gender'] = df_train.apply(designate_sex, axis=1)
df_model = df_train[df_train['gender'].notna()].copy()

# Wybór cech (features) i zmienna docelowa (target)
X_train3 = df_model[features3]
y_train3 = df_model['gender']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_cat3)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

model_pipeline.fit(X_train3, y_train3)

# Predykcja płci dzieci - Model 4
features4 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat4 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

df_train['male_children'] = df_train['total_male'].apply(lambda x: max(x - 1, 0))
df_train['female_children'] = df_train['total_female'].apply(lambda x: max(x - 1, 0))

df_filtered = df_train[(df_train['total_male'] > 0) & (df_train['total_female'] > 0)].copy()
df_children_model = df_filtered[df_filtered['travel_with'] == 'Spouse and Children'].copy()

# Przygotowanie macierzy cech
X_train_child = pd.get_dummies(df_children_model[features4], columns=features_cat4)

# Przygotowanie macierzy target – dwie kolumny: liczba dzieci mężczyzn i dzieci kobiet
y_train_child = df_children_model[['male_children', 'female_children']]

# Inicjalizacja modelu
multioutput_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

# Trenowanie modelu
multioutput_rf.fit(X_train_child, y_train_child)

# Predykcja podziału dzieci oraz dorosłego - Model 5
features5 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat5 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

df_filtered = df_train[(df_train['total_male'] + df_train['total_female'] > 1)].copy()
df_children_only_model = df_filtered[df_filtered['travel_with'] == 'Children'].copy()

X_train_child_only = pd.get_dummies(df_children_only_model[features5], columns=features_cat5)
y_train_child_only = df_children_only_model[['total_male', 'total_female']]

multioutput_rf_children_only = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

multioutput_rf_children_only.fit(X_train_child_only, y_train_child_only)

# Predykcja podziału przyjaciół - Model 6
features6 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat6 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

df_filtered = df_train[(df_train['total_male'] + df_train['total_female'] > 1)].copy()
df_friend_model = df_filtered[df_filtered['travel_with'] == 'Friends/Relatives'].copy()

X_train_friends = pd.get_dummies(df_friend_model[features6], columns=features_cat6)
y_train_friends = df_friend_model[['total_male', 'total_female']]

multioutput_rf_friends = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

multioutput_rf_friends.fit(X_train_friends, y_train_friends)

# predykcja kategori  - Model 7
features7 = ['region', 'age_group', 'total_female',
       'total_male', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat7 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']


allowed_categories7 = ['Friends/Relatives', 'Children', 'Spouse and Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp7 = df_train[df_train['travel_with'].isin(allowed_categories7)].copy()
X_train7 = pd.get_dummies(df_train_imp7[features7], columns=features_cat7)
y_train7 = df_train_imp7['travel_with']
# Trenowanie modelu
rf7 = RandomForestClassifier(n_estimators=100, random_state=42)
rf7.fit(X_train7, y_train7)

# Predykcja podziału dla wycieczki widmo - Model 8
features8 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat8 = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

df_filtered_phantom = df_train[(df_train['total_male'] + df_train['total_female'] > 1)].copy()
df_phantom_model = df_filtered_phantom[(df_filtered_phantom['travel_with'] == 'Friends/Relatives') | (df_filtered_phantom['travel_with'] == 'Spouse and Children') | (df_filtered_phantom['travel_with'] == 'Children')].copy()

X_train_phantom = pd.get_dummies(df_phantom_model[features8], columns=features_cat8)
y_train_phantom = df_phantom_model[['total_male', 'total_female']]

multioutput_rf_phantom = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

multioutput_rf_phantom.fit(X_train_phantom, y_train_phantom)

# Predykcja travel with dla rekordu widmo
allowed_categories8 = ['Children', 'Friends/Relatives', 'Spouse and Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp8 = df_train[df_train['travel_with'].isin(allowed_categories8)].copy()
X_train8 = pd.get_dummies(df_train_imp8[features8], columns=features_cat8)
y_train8 = df_train_imp8['travel_with']
# Trenowanie modelu
rf8 = RandomForestClassifier(n_estimators=100, random_state=42)
rf8.fit(X_train8, y_train8)

# Imputation
for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    # Completing the travel_with field with the estimated value when only one gender participated in the trip and the number of people exceeds one
    if (pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) > 1) &
            ((row['total_male'] == 0 | pd.isna(row['total_male'])) | (row['total_female'] == 0 | pd.isna(row['total_female'])))):

        dummy_df = pd.get_dummies(row[features], columns=features_cat)
        dummy_df.columns = dummy_df.columns.astype(str)
        dummy_df = dummy_df.reindex(columns=X_train.columns, fill_value=0)

        predicted_value = rf.predict(dummy_df)

        df_train.loc[index, 'travel_with'] = predicted_value[0]

    elif (row['total_male'] + row['total_female'] > 1) & (pd.isna(row['travel_with'])):

        dummy_df = pd.get_dummies(row[features], columns=features_cat)
        dummy_df.columns = dummy_df.columns.astype(str)
        dummy_df = dummy_df.reindex(columns=X_train2.columns, fill_value=0)

        predicted_value = rf2.predict(dummy_df)

        print(predicted_value[0])
        df_train.loc[index, 'travel_with'] = predicted_value[0]

    elif (not pd.isna(row['travel_with'])) & (row['total_male'] == 0 | pd.isna(row['total_male'])) & (row['total_female'] == 0 | pd.isna(row['total_female'])):
        print("Brakuje liczby osob")
        persons = round(row['total_cost'] / mean, 0)

        if row['travel_with'] == 'Alone':
            print('predykcja płci')

            input_df = row[features3].to_frame().T

            predicted_value = model_pipeline.predict(input_df)

            if predicted_value[0] == 'Male':
                df_train.loc[index, 'total_male'] = 1
                df_train.loc[index, 'total_female'] = 0
            else:
                df_train.loc[index, 'total_male'] = 0
                df_train.loc[index, 'total_female'] = 1
        elif row['travel_with'] == 'Spouse':
            df_train.loc[index, 'total_male'] = 1
            df_train.loc[index, 'total_female'] = 1
        elif row['travel_with'] == 'Spouse and Children':
            df_train.loc[index, 'total_male'] = 1
            df_train.loc[index, 'total_female'] = 1
            if persons < 3:
                persons = 3
            print(f"Predykcja płci dziecka, przy {persons} wszystkich osobach")
            persons = persons - 2

            # Przygotowanie danych wejściowych dla modelu
            dummy_df = pd.get_dummies(row[features3], columns=features_cat3)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train_child.columns, fill_value=0)

            # Surowe predykcje liczby dzieci dla obu płci
            pred = multioutput_rf.predict(dummy_df)
            pred_male, pred_female = pred[0, 0], pred[0, 1]

            # Skalowanie predykcji do znanej liczby dzieci (persons)
            pred_sum = pred_male + pred_female
            if pred_sum == 0:
                ratio_male = 0.5  # zabezpieczenie, gdyby suma była zerowa
            else:
                ratio_male = pred_male / pred_sum
            ratio_female = 1 - ratio_male

            # Obliczenie ostatecznej liczby dzieci danej płci
            male_children_final = round(persons * ratio_male)
            female_children_final = persons - male_children_final

            # Dodanie przewidywanej liczby dzieci do dorosłych
            df_train.loc[index, 'total_male'] += male_children_final
            df_train.loc[index, 'total_female'] += female_children_final

            print(f"Predykcja: {male_children_final} chłopców oraz {female_children_final} dziewczyn, przy {persons} żądanych osobach")


        elif row['travel_with'] == 'Children':
            print("Predykcja płci dziecka")

            if persons < 2:
                persons = 2

            # Przygotowanie danych wejściowych dla modelu
            dummy_df = pd.get_dummies(row[features5], columns=features_cat5)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train_child_only.columns, fill_value=0)

            # Surowe predykcje liczby dzieci dla obu płci
            pred = multioutput_rf_children_only.predict(dummy_df)
            pred_male, pred_female = pred[0, 0], pred[0, 1]

            # Skalowanie predykcji do znanej liczby osób (persons)
            pred_sum = pred_male + pred_female
            if pred_sum == 0:
                ratio_male = 0.5  # zabezpieczenie, gdyby suma była zerowa
            else:
                ratio_male = pred_male / pred_sum
            ratio_female = 1 - ratio_male

            # Obliczenie ostatecznej liczby dzieci danej płci
            male_final = round(persons * ratio_male)
            female_final = persons - male_final

            # Przypisanie przewidywanej liczby dzieci
            df_train.loc[index, 'total_male'] = male_final
            df_train.loc[index, 'total_female'] = female_final

            print(f"Predykcja: {male_final} chłopców oraz {female_final} dziewczyn, przy {persons} żądanych osobach")

        elif row['travel_with'] == 'Friends/Relatives':
            print("Predykcja podziału")

            if persons < 2:
                persons = 2

            # Przygotowanie danych wejściowych dla modelu
            dummy_df = pd.get_dummies(row[features6], columns=features_cat6)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train_friends.columns, fill_value=0)

            # Surowe predykcje liczby osób dla obu płci
            pred = multioutput_rf_friends.predict(dummy_df)
            pred_male, pred_female = pred[0, 0], pred[0, 1]

            # Skalowanie predykcji do znanej liczby osób (persons)
            pred_sum = pred_male + pred_female
            if pred_sum == 0:
                ratio_male = 0.5  # zabezpieczenie, gdyby suma była zerowa
            else:
                ratio_male = pred_male / pred_sum
            ratio_female = 1 - ratio_male

            # Obliczenie ostatecznej liczby osób danej płci
            male_final = round(persons * ratio_male)
            female_final = persons - male_final

            # Przypisanie przewidywanej liczby osób
            df_train.loc[index, 'total_male'] = male_final
            df_train.loc[index, 'total_female'] = female_final

            print(f"Predykcja: {male_final} mężczyzn oraz {female_final} kobiet, przy {persons} żądanych osobach")
        else:
            print(f"{row['travel_with']}"
              f"\n{row['total_male']}"
              f"\n{row['total_female']}")

    elif pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 0):
        persons = round(row['total_cost'] / mean, 0)

        if persons <= 1:
            print('predykcja płci')
            df_train.loc[index, 'travel_with'] = 'Alone'

            input_df = row[features3].to_frame().T

            predicted_value = model_pipeline.predict(input_df)

            if predicted_value[0] == 'Male':
                df_train.loc[index, 'total_male'] = 1
                df_train.loc[index, 'total_female'] = 0
            else:
                df_train.loc[index, 'total_male'] = 0
                df_train.loc[index, 'total_female'] = 1

        else:
            print('predykcja płci i rozłożenia oraz kategorii travel_with')

            # predykcja podziału osób
            print(f"Predykcja: {persons} osob")

            # Przygotowanie danych wejściowych dla modelu
            dummy_df = pd.get_dummies(row[features8], columns=features_cat8)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train_phantom.columns, fill_value=0)

            # Surowe predykcje liczby osób dla obu płci
            pred = multioutput_rf_phantom.predict(dummy_df)
            pred_male, pred_female = pred[0, 0], pred[0, 1]

            # Skalowanie predykcji do znanej liczby osób (persons)
            pred_sum = pred_male + pred_female
            if pred_sum == 0:
                ratio_male = 0.5  # zabezpieczenie, gdyby suma była zerowa
            else:
                ratio_male = pred_male / pred_sum
            ratio_female = 1 - ratio_male

            # Obliczenie ostatecznej liczby osób danej płci
            male_final = round(persons * ratio_male)
            female_final = persons - male_final

            # Przypisanie przewidywanej liczby osób
            df_train.loc[index, 'total_male'] = male_final
            df_train.loc[index, 'total_female'] = female_final

            row['total_male'] = male_final
            row['total_female'] = female_final

            print(f"Predykcja: {male_final} mężczyzn oraz {female_final} kobiet, przy {persons} żądanych osobach")

            # predykcja kategorii
            dummy_df = pd.get_dummies(row[features8], columns=features_cat8)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train8.columns, fill_value=0)

            predicted_value = rf8.predict(dummy_df)

            df_train.loc[index, 'travel_with'] = predicted_value[0]


    elif pd.isna(row['travel_with']):
        print(f"{row['travel_with']}"
              f"\n{row['total_male']}"
              f"\n{row['total_female']}")


mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])

count3 = df_train.isna().any(axis=1).sum()

print("Początkowa liczba wierszy z brakującymi wartościami:", count)
print("Liczba wierszy z brakującymi wartościami po uzupełnieniu braków w kolumnie most_impressing:", count2)
print("Końcowa liczba wierszy z brakującymi wartościami:", count3)

for column in df_train.columns:
    empty_count = df_train[column].isna().sum()

    if empty_count > 0:
        print(f"Column '{column}' has {empty_count} empty fields (NaN).")

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    #print(row)
    pass

unique_countries_nan = df_train[df_train['region'].isna()]['country'].unique()
print("Państwa bez regionu: ", unique_countries_nan)


df_train.drop(["most_impressing", "gender"], axis=1, inplace=True)
# df_train.drop(["gender"], axis=1, inplace=True)
df_train["total_people"] = df_train["total_male"] + df_train["total_female"]
df_train['night_total'] = df_train['night_zanzibar'] + df_train['night_mainland']


Ustawienia walidacji krzyżowej

In [ ]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
torch.manual_seed(seed)


n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)


Tablice do przechowywania meta-features

In [ ]:
oof_preds_xgb = np.zeros(len(df_train))
oof_preds_cat = np.zeros(len(df_train))
oof_preds_lgb = np.zeros(len(df_train))
meta_features = np.zeros((len(df_train), 3))


Przygotowanie danych

In [ ]:
print(df_train.columns)
# Konwersja kolumn `object` na `category`
for col in df_train.select_dtypes(include=['object']).columns:
    df_train[col] = df_train[col].astype('category')
    new_col = "cat_" + col
    df_train.rename(columns={col: new_col}, inplace=True)

print(df_train.columns)
# Przygotowanie danych wejściowych
X = df_train.drop(columns=["total_cost", "cat_ID", 'total_cost_per_person', 'male_children', 'female_children', "cat_country"])  # Dane wejściowe
y = df_train["total_cost"]
print(X.columns)
cat_features = [col for col in X.columns if col.startswith('cat_')]

# Podział na zbiory treningowy i walidacyjny
if is_test_iteration:
    X, X_valid_end, y, y_valid_end = train_test_split(X, y, test_size=0.2, random_state=42)
    y = y.reset_index(drop=True)

print(f"Liczba wierszy w zbiorze danych treningowych: {len(X)}")
print(f"Liczba wierszy w zbiorze wartości: {len(y)}")


Dostrajanie XGBoost

In [ ]:
def objective(trial):
    # Zakresy hiperparametrów
    params = {
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        # 'gamma': trial.suggest_float('gamma', 0, 10),
        # 'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),

        'random_state': seed,
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.015, 0.17, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 2),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.5),
        'verbose': 0,
        'enable_categorical': True
    }

    # Inicjalizacja modelu z parametrami
    model = XGBRegressor(**params)

    # KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    # Obliczenie metryki
    scores = cross_val_score(
        model,
        X,
        y,
        cv=kf,
        scoring='neg_mean_absolute_error'
    )

    mae = -scores.mean()

    return mae


# Tworzenie i optymalizacja dla XGBoost
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150, show_progress_bar=True)

print("Najlepsza wartość MAE:", study.best_value)
print("Najlepsze parametry:", study.best_params)


Dostrajanie CatBoost

In [ ]:
# Strojenie modelu CatBoost
def objective_catboost(trial):
    params = {
        # 'iterations': trial.suggest_int('iterations', 100, 1000),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        # 'depth': trial.suggest_int('depth', 3, 10),
        # 'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        # 'border_count': trial.suggest_int('border_count', 32, 255),

        'leaf_estimation_iterations': 1,
        'iterations': trial.suggest_int('iterations', 150, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.15, log=True),
        'depth': trial.suggest_int('depth', 3, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 6.0),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_state': seed,
        'verbose': False,
        'cat_features': cat_features,
        'boosting_type': 'Ordered',
        'task_type': 'GPU'
    }
    model = CatBoostRegressor(**params)

    # KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores = cross_val_score(
        model, X, y, cv=kf, scoring='neg_mean_absolute_error'
    )
    mae = -scores.mean()
    return mae


# Tworzenie i optymalizacja dla CatBoost
study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_catboost, n_trials=5, show_progress_bar=True)

print("Najlepsza wartość MAE dla CatBoost:", study_catboost.best_value)
print("Najlepsze parametry dla CatBoost:", study_catboost.best_params)


Dostrajanie LightGBM

In [ ]:
# Strojenie modelu LightGBM
def objective_lightgbm(trial):
    params = {
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # 'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
        # 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        # 'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),

        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'min_child_samples': trial.suggest_int('min_child_samples', 8, 40),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
        'random_state': seed
    }
    model = LGBMRegressor(**params)

    # KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores = cross_val_score(
        model, X, y, cv=kf, scoring='neg_mean_absolute_error'
    )
    mae = -scores.mean()
    return mae


# Tworzenie i optymalizacja dla LightGBM
study_lightgbm = optuna.create_study(direction='minimize')
study_lightgbm.optimize(objective_lightgbm, n_trials=250, show_progress_bar=True)

print("Najlepsza wartość MAE dla LightGBM:", study_lightgbm.best_value)
print("Najlepsze parametry dla LightGBM:", study_lightgbm.best_params)


Dostrajanie Ada Boost

In [ ]:
from sklearn.tree import DecisionTreeRegressor


def objective_adaboost(trial):
    # Parametry AdaBoost
    # params = {
    #     # 'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    #     # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
    #
    #     'n_estimators': trial.suggest_int('n_estimators', 50, 300),
    #     'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.6, log=True),
    #     'loss': trial.suggest_categorical('loss', ['linear', 'square', 'exponential']),
    #     'random_state': seed
    # }

    # Strojenie parametrów bazowego drzewa
    base_params = {
        'max_depth': trial.suggest_int('base_max_depth', 2, 8),
        'min_samples_split': trial.suggest_int('base_min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('base_min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical(
            'base_max_features', [None, 'sqrt', 'log2']
        ),
        'min_weight_fraction_leaf': trial.suggest_float(
            'min_weight_fraction_leaf', 0.0, 0.5
        ),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.1),
        'random_state': seed
    }
    base_est = DecisionTreeRegressor(**base_params)

    # Strojenie AdaBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
        'loss': trial.suggest_categorical(
            'loss', ['linear', 'square', 'exponential']
        ),
        'estimator': base_est,
        'random_state': seed
    }


    num_features = [col for col in X.columns if col not in cat_features]

    # Preprocessor (OneHot na kategoriach, passtrough na liczbowych)
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', num_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
        ]
    )

    # Pipeline: preprocess + model
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', AdaBoostRegressor(**params))
    ])

    # KFold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores = cross_val_score(
        model, X, y, cv=kf, scoring='neg_mean_absolute_error'
    )
    mae = -scores.mean()
    return mae

# Tworzenie i optymalizacja dla AdaBoostRegressor z pipeline
study_adaboost = optuna.create_study(direction='minimize')
study_adaboost.optimize(objective_adaboost, n_trials=150, show_progress_bar=True)

print("Najlepsza wartość MAE dla AdaBoost:", study_adaboost.best_value)
print("Najlepsze parametry dla AdaBoost:", study_adaboost.best_params)



Dostrajanie HGBM

In [ ]:
cat_idx = [X.columns.get_loc(col) for col in cat_features]

def objective_histgbm(trial):
    params = {
        # "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        # "max_iter": trial.suggest_int("max_iter", 100, 1000),
        # "max_depth": trial.suggest_int("max_depth", 3, 16),
        # "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 5.0),
        # "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 15, 150),
        # "min_samples_leaf": trial.suggest_int("min_samples_leaf", 5, 30),

        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15, log=True),
        "max_iter": trial.suggest_int("max_iter", 100, 600),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 2.0),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 16, 60),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 70),
        "random_state": seed,
        "categorical_features": cat_idx if len(cat_idx) > 0 else None
    }
    model = HistGradientBoostingRegressor(**params)

    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores = cross_val_score(
        model, X, y, cv=kf, scoring='neg_mean_absolute_error'
    )
    mae = -scores.mean()
    return mae

# Optymalizacja
study_histgbm = optuna.create_study(direction='minimize')
study_histgbm.optimize(objective_histgbm, n_trials=50, show_progress_bar=True)

print("Najlepsza wartość MAE dla HistGBM:", study_histgbm.best_value)
print("Najlepsze parametry dla HistGBM:", study_histgbm.best_params)


Definicje modeli bazowych i ich parametrów

In [ ]:

xgb_params_manual = {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': seed,
              'enable_categorical': True, 'verbose': 1, 'use_label_encoder': False, 'eval_metric': 'mae'}
cat_params_manual = {'n_estimators': 500, 'learning_rate': 0.05, 'random_state': seed, 'verbose': 1}
lgb_params_manual = {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 6, 'random_state': seed, 'verbose': 1}

xgb_params = {**study.best_params, 'enable_categorical': True, 'verbose': 1}
cat_params = {**study_catboost.best_params, 'verbose': 1, 'leaf_estimation_iterations': 1, 'boosting_type': 'Ordered' }
lgb_params = {**study_lightgbm.best_params, 'verbose': 1}
ada_params = {**study_adaboost.best_params}
hgbm_params = {**study_histgbm.best_params}

# Parametry drzewa DecisionTreeRegressor
tree_param_names = [
    'max_depth', 'min_samples_split', 'min_samples_leaf',
    'min_weight_fraction_leaf', 'max_features', 'ccp_alpha',
    'criterion', 'splitter', 'random_state', 'max_leaf_nodes',
    'min_impurity_decrease'
]

# Parametry AdaBoostRegressor
adaboost_param_names = [
    'estimator', 'n_estimators', 'learning_rate', 'loss', 'random_state'
]

# Inicjalizacja pustych słowników
base_params = {}
ada_only_params = {}

# Rozdzielanie parametrów
for param, value in ada_params.items():
    if param.startswith('base_'):
        # Konwersja nazwy parametru drzewa (usuwanie prefiksu 'base_')
        base_key = param.replace('base_', '')
        base_params[base_key] = value
    elif param in tree_param_names:
        # Parametr drzewa bez prefiksu
        base_params[param] = value
    elif param in adaboost_param_names:
        # Parametr AdaBoosta
        ada_only_params[param] = value
    else:
        # Pozostałe (mogą być istotne dla AdaBoosta)
        ada_only_params[param] = value

# Tworzenie drzewa bazowego
base_estimator = DecisionTreeRegressor(**base_params)

# Ustawianie estymatora dla AdaBoosta
ada_only_params['estimator'] = base_estimator

# Usuwanie potencjalnie konfliktujących parametrów
for param in ['min_weight_fraction_leaf', 'max_features', 'ccp_alpha']:
    if param in ada_only_params:
        del ada_only_params[param]



Inicjalizacja modeli

In [ ]:
num_features = [col for col in X.columns if col not in cat_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

ada_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', AdaBoostRegressor(**ada_only_params))
])

xgb_model = xgb.XGBRegressor(**xgb_params)
cat_model = CatBoostRegressor(**cat_params)
lgb_model = LGBMRegressor(**lgb_params)
# ada_model = AdaBoostRegressor(**ada_params)
hgbm_model = HistGradientBoostingRegressor(**hgbm_params, categorical_features=cat_idx)

models = [('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model), ('ada', ada_pipeline), ('hgbm', hgbm_model)]

models_dict = {
    "LGBM": lgb_model,
    "XGBoost": xgb_model,
    "CatBoost": cat_model,
    # "AdaBoost": ada_pipeline
    # "HistGBM": hgbm_model
}


oof_preds = {
    'xgb': np.zeros(len(X)),
    'cat': np.zeros(len(X)),
    'lgb': np.zeros(len(X)),
    'ada': np.zeros(len(X)),
    'hgbm': np.zeros(len(X)),
    # 'mean': np.zeros(len(X)),
    # 'total_people': np.zeros(len(X)),
    # 'night_total': np.zeros(len(X)),
}


In [ ]:
mae_scores = []

meta_model_weights = {
    'xgb': 0,
    'cat': 0,
    'lgb': 0,
    'ada': 0,
    'hgbm': 0
}


for train_idx, val_idx in kf.split(X):
    X_train, X_valid = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[val_idx]

    # Trening CatBoost
    cat_model.fit(X_train, y_train, cat_features=cat_features)
    cat_preds = cat_model.predict(X_valid)
    oof_preds['cat'][val_idx] = cat_preds
    mae_scores.append(f"CatBoost fold MAE: {mean_absolute_error(y_valid, cat_preds):.4f}")
    meta_model_weights['cat'] += mean_absolute_error(y_valid, cat_preds)

    # Trening XGBoost
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_valid)
    oof_preds['xgb'][val_idx] = xgb_preds
    mae_scores.append(f"XGBoost fold MAE: {mean_absolute_error(y_valid, xgb_preds):.4f}")
    meta_model_weights['xgb'] += mean_absolute_error(y_valid, xgb_preds)

    # Trening LightGBM
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_valid)
    oof_preds['lgb'][val_idx] = lgb_preds
    mae_scores.append(f"LightGBM fold MAE: {mean_absolute_error(y_valid, lgb_preds):.4f}")
    meta_model_weights['lgb'] += mean_absolute_error(y_valid, lgb_preds)

    # Trening AdaBoostRegressor
    ada_pipeline.fit(X_train, y_train)
    ada_preds = ada_pipeline.predict(X_valid)
    oof_preds['ada'][val_idx] = ada_preds
    mae_scores.append(f"AdaBoost fold MAE: {mean_absolute_error(y_valid, ada_preds):.4f}")
    meta_model_weights['ada'] += mean_absolute_error(y_valid, ada_preds)

    hgbm_model.fit(X_train, y_train)
    histgbm_preds = hgbm_model.predict(X_valid)
    oof_preds['hgbm'][val_idx] = histgbm_preds
    mae_scores.append(f"HistGBM fold MAE: {mean_absolute_error(y_valid, histgbm_preds):.4f}")
    meta_model_weights['hgbm'] += mean_absolute_error(y_valid, histgbm_preds)

    # oof_preds['mean'][val_idx] = np.mean([cat_preds, xgb_preds, lgb_preds, ada_preds, histgbm_preds], axis=0)

    # zapis kolumn total_nights i total_person
    # oof_preds['night_total'][val_idx] = X_valid['night_total'].values
    # oof_preds['total_people'][val_idx] = X_valid['total_people'].values

meta_model_weights['xgb'] = meta_model_weights['xgb'] / n_splits
meta_model_weights['cat'] = meta_model_weights['cat'] / n_splits
meta_model_weights['lgb'] = meta_model_weights['lgb'] / n_splits
meta_model_weights['hgbm'] = meta_model_weights['hgbm'] / n_splits
meta_model_weights['ada'] = meta_model_weights['ada'] / n_splits

order = ['xgb', 'cat', 'lgb', 'ada', 'hgbm']
mae_vals = np.array([meta_model_weights[m] for m in order], dtype=np.float32)

# Odwrotność błędu, lepszy model ma większą wagę
inv = 1.0 / (mae_vals + 1e-8)   # dodajemy epsilon, by uniknąć dzielenia przez 0

# Suma wag = 1
weights = inv / inv.sum()


print("Weights: ")
print(meta_model_weights)

meta_features = np.column_stack((
    oof_preds['xgb'],  # kolumna z predykcji XGBoost
    oof_preds['cat'],  # kolumna z predykcji CatBoost
    oof_preds['lgb'],  # kolumna z predykcji LightGBM
    oof_preds['ada'],  # kolumna z predykcji AdaBoost
    oof_preds['hgbm'],
    # oof_preds['mean'],
    # oof_preds['total_people'],
    # oof_preds['night_total']
))

cat_model = CatBoostRegressor(**cat_params)
cat_model.fit(X, y, cat_features=cat_features)

xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X, y)

lgb_model = LGBMRegressor(**lgb_params)
lgb_model.fit(X, y)

ada_pipeline.fit(X, y)

hgbm_model = HistGradientBoostingRegressor(**hgbm_params, categorical_features=cat_idx)
hgbm_model.fit(X, y)

for score in mae_scores:
    print(f"Scores for learning meta model: {score}")

if is_test_iteration:
    cat_preds = cat_model.predict(X_valid_end)
    print(f"CatBoost fold MAE full model: {mean_absolute_error(y_valid_end, cat_preds):.4f}")

    xgb_preds = xgb_model.predict(X_valid_end)
    print(f"XGBoost fold MAE full model: {mean_absolute_error(y_valid_end, xgb_preds):.4f}")

    lgb_preds = lgb_model.predict(X_valid_end)
    print(f"LightGBM fold MAE full model: {mean_absolute_error(y_valid_end, lgb_preds):.4f}")

    ada_preds = ada_pipeline.predict(X_valid_end)
    print(f"AdaBoost fold MAE full model: {mean_absolute_error(y_valid_end, ada_preds):.4f}")

    histgbm_preds = hgbm_model.predict(X_valid_end)
    print(f"HistGBM fold MAE full model: {mean_absolute_error(y_valid_end, histgbm_preds):.4f}")



Feature importances

In [ ]:
for name, model in models_dict.items():
    print(f"\nFeature importances for {name}:")
    importances = model.feature_importances_
    fi = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    print(f"\n=== {name} – Top 10 cech ===")
    print(fi.head(10))

    # Wykres top 20
    plt.figure(figsize=(6,6))
    fi.head(20).plot(kind="barh")
    plt.gca().invert_yaxis()
    plt.title(f"{name} – feature_importances")
    plt.xlabel("Importance")
    plt.tight_layout()
    plt.show()


Analiza SHAP

In [ ]:
for name, model in models_dict.items():
    explainer = shap.TreeExplainer(model)
    shap_vals = explainer.shap_values(X)

    # 1) Globalne znaczenie (bar plot)
    fig, ax = plt.subplots(figsize=(8, 4))
    shap.summary_plot(
        shap_vals, X,
        plot_type="bar",
        show=False,
        color_bar_label="Mean(|SHAP value|)"
    )
    ax.set_title(f"Globalne znaczenie cech – {name}", fontsize=14)
    plt.tight_layout()
    plt.show()

    # 2) Beeswarm (punktowy summary plot)
    fig, ax = plt.subplots(figsize=(8, 6))
    shap.summary_plot(
        shap_vals, X,
        show=False
    )
    ax.set_title(f"SHAP Beeswarm – {name}", fontsize=14)
    plt.tight_layout()
    plt.show()

    # 3) Dependence plot dla najważniejszej cechy
    mean_abs_shap = np.abs(shap_vals).mean(axis=0)
    top_feat = X.columns[mean_abs_shap.argmax()]

    # rysujemy dependence plot
    plt.figure(figsize=(6,4))
    shap.dependence_plot(
        top_feat,
        shap_vals,
        X,
        show=False,
        dot_size=50
    )
    plt.title(f"Dependence plot dla '{top_feat}' – {name}", fontsize=14)
    plt.tight_layout()
    plt.show()




In [ ]:


"""
def build_meta_model(input_dim, n_neurons1, n_neurons2, n_neurons3, n_neurons4, n_neurons5,
 dropout_rate, learning_rate, weight_decay):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(n_neurons1, input_dim=input_dim, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(n_neurons2, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(n_neurons3, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(n_neurons4, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(n_neurons5, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    # AdamW zamiast Adam
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(optimizer=optimizer, loss='mae')
    return model

def objective(trial):
    # Zakresy hiperparametrów
    n_neurons1 = trial.suggest_int("n_neurons1", 64, 128)
    n_neurons2 = trial.suggest_int("n_neurons2", 32, 96)
    n_neurons3 = trial.suggest_int("n_neurons3", 16, 64)
    n_neurons4 = trial.suggest_int("n_neurons4", 8, 32)
    n_neurons5 = trial.suggest_int("n_neurons5", 4, 16)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    epochs = 100
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 96, 128])
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-2)

    # Walidacji krzyżowa
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []

    for train_idx, val_idx in kf.split(meta_features):
        X_train, X_val = meta_features[train_idx], meta_features[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = build_meta_model(meta_features.shape[1], n_neurons1, n_neurons2, n_neurons3, n_neurons4, n_neurons5, dropout_rate, learning_rate, weight_decay)
        reduce_lr = ReduceLROnPlateau(
            monitor='loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        )

        early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

        # W wywołaniu fit:
        model.fit(X_train, y_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  verbose=0,
                  callbacks=[early_stop, reduce_lr])


        mae = model.evaluate(X_val, y_val, verbose=0)
        mae_scores.append(mae)
    print(f"MAE: {np.mean(mae_scores)}")
    # Zwracamy średnią wartość błędu
    return np.mean(mae_scores)


# Uruchomienie optymalizacji
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Najlepsze hiperparametry: ", study.best_params)

# Trenowanie finalnego modelu z najlepszymi hiperparametrami
best_params = study.best_params
meta_model = build_meta_model(meta_features.shape[1],
                               best_params["n_neurons1"],
                               best_params["n_neurons2"],
                               best_params["n_neurons3"],
                               best_params["n_neurons4"],
                               best_params["n_neurons5"],
                               best_params["dropout_rate"],
                               best_params["learning_rate"],
                               best_params["weight_decay"])
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
meta_model.fit(meta_features, y, epochs=100, batch_size=best_params["batch_size"], verbose=1, callbacks=[early_stop])
"""


def build_meta_model(input_dim, architecture, dropout_rate, learning_rate, weight_decay, activation='relu', use_weight_layer=False, weights=None):
    model = tf.keras.Sequential()

    # Warstwa normalizacji na wejściu
    model.add(tf.keras.layers.BatchNormalization(input_shape=(input_dim,)))

    # Warstwa ważenia
    if use_weight_layer:
        if weights is None or len(weights) != input_dim:
            raise ValueError("Musisz podać 'weights' o długości input_dim")
        # stwórz tensor wag i wymuś broadcast po batchu
        w_tensor = tf.constant(weights, shape=(1, input_dim), dtype=tf.float32)
        model.add(
            tf.keras.layers.Lambda(
                lambda x: x * w_tensor,
                name='apply_weights'
            )
        )

    # Warstwy
    for units in architecture:
        model.add(tf.keras.layers.Dense(units, activation=activation))
        if dropout_rate > 0:
            model.add(tf.keras.layers.Dropout(dropout_rate))

    # Wyjściowa warstwa regresji
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    # AdamW
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        clipnorm=1.0
    )

    model.compile(
        optimizer=optimizer,
        loss='mae',
        metrics=['mae']
    )
    return model

def objective_nn(trial):
    # Architektura
    n_layers = trial.suggest_int("n_layers", 2, 2)  # zakres warstw

    # Architektura zwężająca się - więcej neuronów na początku, mniej na końcu
    architecture = []
    for i in range(n_layers):
        if i == 0:
            units = trial.suggest_int(f"units_layer{i}", 250, 500)
        else:
            # Każda kolejna warstwa ma 30-70% neuronów poprzedniej
            prev_units = architecture[-1]
            min_units = max(16, int(prev_units * 0.35))
            max_units = max(32, int(prev_units * 0.6))
            units = trial.suggest_int(f"units_layer{i}", min_units, max_units)
        architecture.append(units)

    # Hiperparametry
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.4)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.01, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 96, 128])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    activation = trial.suggest_categorical("activation", ["relu", "selu", "elu"])

    # Walidacja krzyżowa
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []

    for train_idx, val_idx in kf.split(meta_features):
        X_train, X_val = meta_features[train_idx], meta_features[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Standardyzacja cech
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = build_meta_model(
            meta_features.shape[1],
            architecture,
            dropout_rate,
            learning_rate,
            weight_decay,
            activation,
            use_weight_layer=True,
            weights=weights.tolist()
        )

        # callbacki i monitorowanie
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=0
        )

        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=0
        )

        history = model.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=200,
            batch_size=batch_size,
            verbose=0,
            callbacks=[early_stop, reduce_lr]
        )

        # Ocena modelu na zbiorze walidacyjnym
        mae = model.evaluate(X_val_scaled, y_val, verbose=0)[0]
        mae_scores.append(mae)

    mean_mae = np.mean(mae_scores)
    print(f"Architektura: {architecture}, MAE: {mean_mae:.4f}")
    return mean_mae

# Uruchomienie optymalizacji
study = optuna.create_study(direction="minimize")
study.optimize(objective_nn, n_trials=5)

print(f"Najlepsza wartość MAE: {study.best_value:.4f}")
print("Najlepsze hiperparametry:", study.best_params)

# Trenowanie finalnego modelu z najlepszymi hiperparametrami
best_params = study.best_params

# Architektura z najlepszych parametrów
best_architecture = []
for i in range(best_params["n_layers"]):
    best_architecture.append(best_params[f"units_layer{i}"])

# Standardyzacja całego zbioru dla finalnego modelu
scaler = StandardScaler()
meta_features_scaled = scaler.fit_transform(meta_features)

# Budowa i trenowanie najlepszego modelu
meta_model = build_meta_model(
    meta_features.shape[1],
    best_architecture,
    best_params["dropout_rate"],
    best_params["learning_rate"],
    best_params["weight_decay"],
    best_params["activation"],
    use_weight_layer=True,
    weights=weights.tolist()
)

# Callbacki dla finalnego treningu
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)
early_stop = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

# Trenowanie finalnego modelu
history = meta_model.fit(
    meta_features_scaled, y,
    epochs=200,
    batch_size=best_params["batch_size"],
    verbose=1,
    callbacks=[early_stop, reduce_lr]
)

# Wizualizacja procesu uczenia
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.title('Loss podczas treningu')
plt.ylabel('MAE')
plt.xlabel('Epoka')

if 'val_loss' in history.history:
    plt.subplot(1, 2, 2)
    plt.plot(history.history['val_loss'])
    plt.title('Walidacyjny loss')
    plt.ylabel('MAE')
    plt.xlabel('Epoka')

plt.tight_layout()
plt.show()


In [ ]:
## 2. Optymalizacja Ridge

def objective_ridge(trial):
    from sklearn.linear_model import Ridge
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
    model = Ridge(alpha=alpha, random_state=seed)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_ridge = optuna.create_study(direction="minimize")
study_ridge.optimize(objective_ridge, n_trials=50)

print("Ridge best MAE:", study_ridge.best_value)
print("Ridge best params:", study_ridge.best_params)


## 3. Optymalizacja Lasso

def objective_lasso(trial):
    from sklearn.linear_model import Lasso
    alpha = trial.suggest_float("alpha", 1e-4, 1.0, log=True)
    model = Lasso(alpha=alpha, random_state=seed, max_iter=5000)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_lasso = optuna.create_study(direction="minimize")
study_lasso.optimize(objective_lasso, n_trials=50)

print("Lasso best MAE:", study_lasso.best_value)
print("Lasso best params:", study_lasso.best_params)


## 4. Optymalizacja SVR

# def objective_svr(trial):
#     from sklearn.svm import SVR
#     C = trial.suggest_float("C", 0.1, 100.0, log=True)
#     epsilon = trial.suggest_float("epsilon", 0.01, 1.0, log=True)
#     gamma = trial.suggest_categorical("gamma", ["scale", "auto"])
#     model = SVR(kernel="rbf", C=C, epsilon=epsilon, gamma=gamma)
#     scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
#     return -scores.mean()
#
#
# study_svr = optuna.create_study(direction="minimize")
# study_svr.optimize(objective_svr, n_trials=50)
#
# print("SVR best MAE:", study_svr.best_value)
# print("SVR best params:", study_svr.best_params)



# Funkcja celu dla optymalizacji
def objective_elastic(trial):
    alpha = trial.suggest_float("alpha", 1e-5, 1.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.01, 0.99)
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=seed, max_iter=5000)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_elastic = optuna.create_study(direction="minimize")
study_elastic.optimize(objective_elastic, n_trials=50)
print("ElasticNet best MAE:", study_elastic.best_value)
print("ElasticNet best params:", study_elastic.best_params)



# Funkcja celu dla optymalizacji
def objective_lgbm(trial):
    params = {
        "objective": "regression",
        "metric": "mae",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "random_state": seed
    }

    model = lgb.LGBMRegressor(**params)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_lgbm = optuna.create_study(direction="minimize")
study_lgbm.optimize(objective_lgbm, n_trials=50)
print("LightGBM best MAE:", study_lgbm.best_value)
print("LightGBM best params:", study_lgbm.best_params)

# Funkcja celu dla optymalizacji
def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "random_seed": seed,
        "verbose": False
    }

    model = CatBoostRegressor(**params)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_catboost = optuna.create_study(direction="minimize")
study_catboost.optimize(objective_catboost, n_trials=50)
print("CatBoost best MAE:", study_catboost.best_value)
print("CatBoost best params:", study_catboost.best_params)


# Funkcja celu dla optymalizacji
def objective_et(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=seed,
        n_jobs=-1
    )

    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_et = optuna.create_study(direction="minimize")
study_et.optimize(objective_et, n_trials=50)
print("ExtraTrees best MAE:", study_et.best_value)
print("ExtraTrees best params:", study_et.best_params)


# Standardyzacja danych
scaler_mlp = StandardScaler()
meta_features_scaled = scaler_mlp.fit_transform(meta_features)

# Funkcja celu dla optymalizacji
def objective_mlp(trial):
    # Architektura sieci
    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden_layers = []
    for i in range(n_layers):
        hidden_layers.append(trial.suggest_int(f"n_units_l{i}", 32, 256))

    # Hiperparametry
    alpha = trial.suggest_float("alpha", 1e-6, 1e-2, log=True)  # Regularyzacja
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-4, 1e-1, log=True)
    activation = trial.suggest_categorical("activation", ["tanh", "relu"])

    model = MLPRegressor(
        hidden_layer_sizes=tuple(hidden_layers),
        alpha=alpha,
        learning_rate_init=learning_rate_init,
        activation=activation,
        max_iter=2000,
        early_stopping=True,
        random_state=seed
    )

    scores = cross_val_score(model, meta_features_scaled, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_mlp = optuna.create_study(direction="minimize")
study_mlp.optimize(objective_mlp, n_trials=50)
print("MLPRegressor best MAE:", study_mlp.best_value)
print("MLPRegressor best params:", study_mlp.best_params)

def objective_huber(trial):
    epsilon = trial.suggest_float("epsilon", 1.1, 2.0)
    alpha = trial.suggest_float("alpha", 1e-6, 1.0, log=True)

    model = HuberRegressor(
        epsilon=epsilon,
        alpha=alpha,
        max_iter=2000
    )

    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()

# Optymalizacja
study_huber = optuna.create_study(direction="minimize")
study_huber.optimize(objective_huber, n_trials=50)
print("HuberRegressor best MAE:", study_huber.best_value)
print("HuberRegressor best params:", study_huber.best_params)


## 5. Optymalizacja Random Forest

def objective_rf(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", [None, "sqrt", "log2"])
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=seed,
        n_jobs=-1
    )
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_rf = optuna.create_study(direction="minimize")
study_rf.optimize(objective_rf, n_trials=50)

print("RF best MAE:", study_rf.best_value)
print("RF best params:", study_rf.best_params)


## 6. Optymalizacja GradientBoosting

def objective_gbr(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    max_features = trial.suggest_categorical("max_features", [None, "sqrt", "log2"])
    model = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        subsample=subsample,
        max_features=max_features,
        random_state=seed
    )
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_gbr = optuna.create_study(direction="minimize")
study_gbr.optimize(objective_gbr, n_trials=50)

print("GBR best MAE:", study_gbr.best_value)
print("GBR best params:", study_gbr.best_params)


## 7. Optymalizacja XGBoost

def objective_xgb(trial):

    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "random_state": seed,
        "verbosity": 0
    }
    model = xgb.XGBRegressor(**params)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=50)

print("XGB best MAE:", study_xgb.best_value)
print("XGB best params:", study_xgb.best_params)


## 8. Optymalizacja KNN

def objective_knn(trial):
    from sklearn.neighbors import KNeighborsRegressor
    n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
    weights = trial.suggest_categorical("weights", ["uniform", "distance"])
    p = trial.suggest_int("p", 1, 5)
    model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p, n_jobs=-1)
    scores = cross_val_score(model, meta_features, y, cv=kf, scoring="neg_mean_absolute_error")
    return -scores.mean()


study_knn = optuna.create_study(direction="minimize")
study_knn.optimize(objective_knn, n_trials=50)

print("KNN best MAE:", study_knn.best_value)
print("KNN best params:", study_knn.best_params)




def evaluate_models(train_X, train_y, valid_X, valid_y, models: dict):
    """
    Trenuje każdy model na (train_X, train_y),
    dokonuje predykcji na (valid_X),
    zwraca:
      - results[name] = MAE na zbiorze walidacyjnym
      - predictions[name] = wektor predykcji dla valid_X
    """
    results = {}
    predictions = {}

    scaler_mlp = StandardScaler()
    train_X_scaled = scaler_mlp.fit_transform(train_X)
    valid_X_scaled = scaler_mlp.transform(valid_X)

    for name, model in models.items():
        if name == "MLP":
            model.fit(train_X_scaled, train_y)
            pred = model.predict(valid_X_scaled)
        else:
            model.fit(train_X, train_y)
            pred = model.predict(valid_X)

        mae = mean_absolute_error(valid_y, pred)
        results[name] = mae
        predictions[name] = pred
        print(f"MAE {name}: {mae:.4f}")
    return results, predictions



# Utworzenie parametrów dla MLPRegressor
mlp_params = study_mlp.best_params.copy()

n_layers = mlp_params.pop('n_layers', 0)

# Tworzenie architektury sieci neuronowej
hidden_layer_sizes = []
for i in range(n_layers):
    if f'n_units_l{i}' in mlp_params:
        hidden_layer_sizes.append(mlp_params.pop(f'n_units_l{i}'))


models = {
    "Ridge": Ridge(**study_ridge.best_params, random_state=seed),
    "Lasso": Lasso(**study_lasso.best_params, random_state=seed, max_iter=5000),
    "RandomForest": RandomForestRegressor(**study_rf.best_params, random_state=seed, n_jobs=-1),
    "GBR": GradientBoostingRegressor(**study_gbr.best_params, random_state=seed),
    "XGBoost": xgb.XGBRegressor(**study_xgb.best_params, random_state=seed),
    "KNN": KNeighborsRegressor(**study_knn.best_params, n_jobs=-1),
    "ElasticNet": ElasticNet(**study_elastic.best_params, random_state=seed, max_iter=5000),
    "LightGBM": lgb.LGBMRegressor(**study_lgbm.best_params, random_state=seed),
    "CatBoost": CatBoostRegressor(**study_catboost.best_params, random_seed=seed, verbose=False),
    "ExtraTrees": ExtraTreesRegressor(**study_et.best_params, random_state=seed, n_jobs=-1),
    "MLP": MLPRegressor(
        hidden_layer_sizes=tuple(hidden_layer_sizes),
        **mlp_params,
        random_state=seed,
        max_iter=2000,
        early_stopping=True
    ),
    "Huber": HuberRegressor(**study_huber.best_params, max_iter=2000)
}

if is_test_iteration:
    preds_xgb = xgb_model.predict(X_valid_end)
    preds_cat = cat_model.predict(X_valid_end)
    preds_lgb = lgb_model.predict(X_valid_end)
    preds_ada = ada_pipeline.predict(X_valid_end)
    preds_hgbm = hgbm_model.predict(X_valid_end)

    meta_features_new = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))

    results, predictions = evaluate_models(
        meta_features, y,
        meta_features_new, y_valid_end,
        models
    )

    names = list(results.keys())
    maes  = list(results.values())

    plt.figure(figsize=(10,5))
    plt.bar(results.keys(), results.values(), color="skyblue")
    plt.xticks(rotation=45)
    plt.ylabel("MAE")
    plt.title("Porównanie MAE modeli")
    plt.tight_layout()
    plt.show()

    for name, pred in predictions.items():
        plt.figure(figsize=(6,6))
        plt.scatter(y_valid_end, pred, alpha=0.6)
        minv, maxv = y_valid_end.min(), y_valid_end.max()
        plt.plot([minv, maxv], [minv, maxv], "r--", lw=1)
        plt.xlabel("y rzeczywiste (walidacja)")
        plt.ylabel("y_pred")
        plt.title(f"{name}: walidacja y vs y_pred")
        plt.tight_layout()
        plt.show()


In [ ]:

# Meta model sieci neuronowej łączący wszystkie meta modele
# 1. Funkcja do generowania predykcji out-of-fold (OOF) dla każdego meta modelu
def generate_oof_predictions(models, X, y, n_splits=5):
    """
    Generuje predykcje out-of-fold dla każdego modelu.

    Args:
        models (dict): Słownik modeli (nazwa: model)
        X (np.array): Cechy
        y (np.array): Wartości docelowe
        n_splits (int): Liczba podziałów w walidacji krzyżowej

    Returns:
        tuple: (oof_predictions, model_mae_scores)
            - oof_predictions (np.array): Macierz predykcji OOF dla każdego modelu
            - model_mae_scores (dict): Słownik z wynikami MAE dla każdego modelu
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    # Inicjalizacja macierzy out-of-fold
    n_samples = X.shape[0]
    n_models = len(models)
    oof_predictions = np.zeros((n_samples, n_models))
    model_mae_scores = {}

    # Dla każdego podziału w walidacji krzyżowej
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx], y.iloc[val_idx] if hasattr(y, 'iloc') else y[val_idx]

        print(f"Fold {fold+1}/{n_splits}")

        # Standardyzacja danych dla modeli wymagających skalowania
        scaler_oof = StandardScaler()
        X_train_scaled = scaler_oof.fit_transform(X_train)
        X_val_scaled = scaler_oof.transform(X_val)


        # Dla każdego modelu
        for i, (name, model) in enumerate(models.items()):
            print(f"  Trenowanie {name}...")

            # Trenowanie modelu
            if name == "NN":
                # Callbacki dla finalnego treningu
                reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)
                early_stop = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

                # Trenowanie finalnego modelu
                model.fit(
                    X_train_scaled, y_train,
                    epochs=200,
                    batch_size=best_params["batch_size"],
                    verbose=1,
                    callbacks=[early_stop, reduce_lr]
                )

                val_pred = model.predict(X_val_scaled, verbose=0)
            elif name == "MLP":
                model.fit(X_train_scaled, y_train)
                val_pred = model.predict(X_val_scaled)
            else:
                model.fit(X_train, y_train)
                val_pred = model.predict(X_val)

            # Upewnienie się, że predykcje mają odpowiedni kształt, konwersja na płaską tablicę
            val_pred = np.asarray(val_pred).flatten()

            # Zapisanie predykcji OOF
            oof_predictions[val_idx, i] = val_pred

            # Obliczenie MAE na zbiorze walidacyjnym
            mae = mean_absolute_error(y_val, val_pred)

            # Aktualizacja średniego MAE dla modelu
            if name not in model_mae_scores:
                model_mae_scores[name] = []
            model_mae_scores[name].append(mae)

            print(f"    MAE dla {name} w fold {fold+1}: {mae:.4f}")

    # Obliczenie średniego MAE dla każdego modelu
    for name in model_mae_scores:
        model_mae_scores[name] = np.mean(model_mae_scores[name])

    return oof_predictions, model_mae_scores


# 2. Budowa finalnego meta modelu wykorzystującego predykcje z modeli bazowych
def build_final_meta_model(input_dim, architecture, dropout_rate, learning_rate, weight_decay,
                          initial_weights=None, use_weight_layer=True, activation='relu'):
    """
    Buduje finalny meta model, który wykorzystuje predykcje z innych modeli.

    Args:
        input_dim (int): Liczba wejściowych cech (liczba modeli bazowych)
        architecture (list): Lista z liczbą neuronów w każdej warstwie ukrytej
        dropout_rate (float): Wartość dropout do regularyzacji
        learning_rate (float): Współczynnik uczenia
        weight_decay (float): Regularyzacja wag
        initial_weights (list, optional): Początkowe wagi do zastosowania dla modeli bazowych
        use_weight_layer (bool): Czy używać warstwy z wagami
        activation (str): Funkcja aktywacji

    Returns:
        tf.keras.Model: Skompilowany model
    """
    model = tf.keras.Sequential()

    # Warstwa normalizacji na wejściu
    model.add(tf.keras.layers.BatchNormalization(input_shape=(input_dim,)))

    # Warstwa ważenia predykcji modeli bazowych
    if use_weight_layer and initial_weights is not None:
        if len(initial_weights) != input_dim:
            raise ValueError(f"Wagi muszą mieć długość {input_dim}, otrzymano {len(initial_weights)}")

        # Stworzenie tensora wag
        w_tensor = tf.constant(initial_weights, shape=(1, input_dim), dtype=tf.float32)

        # Warstwa Lambda z wagami
        model.add(
            tf.keras.layers.Lambda(
                lambda x: x * w_tensor,
                name='model_weights'
            )
        )

    # Warstw ukryte
    for units in architecture:
        model.add(tf.keras.layers.Dense(units, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
        if dropout_rate > 0:
            model.add(tf.keras.layers.Dropout(dropout_rate))

    # Warstwa wyjściowa
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        clipnorm=1.0
    )

    model.compile(
        optimizer=optimizer,
        loss='mae',
        metrics=['mae']
    )

    return model

# 3. Funkcja celu dla optymalizacji Optuna
def objective_final_meta(trial, X, y, initial_weights):
    """
    Funkcja celu dla optymalizacji Optuna.

    Args:
        trial: Trial Optuna
        X (np.array): Macierz predykcji OOF
        y (np.array): Wartości docelowe
        initial_weights (list): Początkowe wagi dla modeli bazowych

    Returns:
        float: Średni MAE z walidacji krzyżowej
    """
    # Parametry do optymalizacji
    n_layers = trial.suggest_int("n_layers", 2, 3)

    # Architektura sieci
    architecture = []
    for i in range(n_layers):
        if i == 0:
            units = trial.suggest_int(f"units_layer{i}", 128, 356)
        else:
            # Każda kolejna warstwa ma mniej neuronów
            prev_units = architecture[-1]
            min_units = max(4, int(prev_units * 0.35))
            max_units = max(8, int(prev_units * 0.6))
            units = trial.suggest_int(f"units_layer{i}", min_units, max_units)
        architecture.append(units)

    # Pozostałe hiperparametry
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.4)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.01, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    activation = trial.suggest_categorical("activation", ["relu", "selu", "elu"])
    use_weight_layer = trial.suggest_categorical("use_weight_layer", [True, False])

    # Walidacja krzyżowa
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    mae_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx], y.iloc[val_idx] if hasattr(y, 'iloc') else y[val_idx]

        # Standardyzacja cech
        scaler_fin = StandardScaler()
        X_train_scaled = scaler_fin.fit_transform(X_train)
        X_val_scaled = scaler_fin.transform(X_val)

        # Budowa modelu
        model = build_final_meta_model(
            X.shape[1],
            architecture,
            dropout_rate,
            learning_rate,
            weight_decay,
            initial_weights if use_weight_layer else None,
            use_weight_layer,
            activation
        )

        # Callbacks
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=0
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=0
        )

        # Trenowanie modelu
        history = model.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=100,
            batch_size=batch_size,
            verbose=0,
            callbacks=[early_stop, reduce_lr]
        )

        # Ocena modelu
        mae = model.evaluate(X_val_scaled, y_val, verbose=0)[1]
        mae_scores.append(mae)

    mean_mae = np.mean(mae_scores)
    print(f"Architektura: {architecture}, MAE: {mean_mae:.4f}")
    return mean_mae

# 4. Główna część - wygenerowanie OOF dla modeli, optymalizacja i trenowanie
mlp_params = study_mlp.best_params.copy()
n_layers = mlp_params.pop('n_layers', 0)
hidden_layer_sizes = []
for i in range(n_layers):
    if f'n_units_l{i}' in mlp_params:
        hidden_layer_sizes.append(mlp_params.pop(f'n_units_l{i}'))

models_dict = {
    "Ridge": Ridge(**study_ridge.best_params, random_state=seed),
    "Lasso": Lasso(**study_lasso.best_params, random_state=seed, max_iter=5000),
    "ElasticNet": ElasticNet(**study_elastic.best_params, random_state=seed, max_iter=5000),
    "RandomForest": RandomForestRegressor(**study_rf.best_params, random_state=seed, n_jobs=-1),
    "GBR": GradientBoostingRegressor(**study_gbr.best_params, random_state=seed),
    "XGBoost": xgb.XGBRegressor(**study_xgb.best_params, random_state=seed),
    "LightGBM": lgb.LGBMRegressor(**study_lgbm.best_params, random_state=seed),
    "CatBoost": CatBoostRegressor(**study_catboost.best_params, random_seed=seed, verbose=False),
    "ExtraTrees": ExtraTreesRegressor(**study_et.best_params, random_state=seed, n_jobs=-1),
    "KNN": KNeighborsRegressor(**study_knn.best_params, n_jobs=-1),
    "Huber": HuberRegressor(**study_huber.best_params, max_iter=2000),
    "MLP": MLPRegressor(
        hidden_layer_sizes=tuple(hidden_layer_sizes),
        **mlp_params,
        random_state=seed,
        max_iter=2000,
        early_stopping=True
    ),
    "NN": meta_model
}


# Generowanie predykcji OOF dla każdego modelu
print("Generowanie predykcji out-of-fold dla modeli bazowych...")
oof_predictions, model_mae_scores = generate_oof_predictions(models_dict, meta_features, y)

# Wizualizacja wyników modeli bazowych
plt.figure(figsize=(12, 6))
model_names = list(model_mae_scores.keys())
mae_values = list(model_mae_scores.values())
sorted_indices = np.argsort(mae_values)
plt.bar(
    range(len(model_names)),
    [mae_values[i] for i in sorted_indices],
    color='skyblue'
)
plt.xticks(
    range(len(model_names)),
    [model_names[i] for i in sorted_indices],
    rotation=45
)
plt.title('Średnie MAE modeli bazowych (out-of-fold)')
plt.ylabel('MAE')
plt.tight_layout()
plt.show()

# Obliczenie wag inicjujących na podstawie MAE
# Im niższe MAE, tym wyższa waga
initial_weights = []
sum_inverse_mae = sum(1.0 / mae for mae in model_mae_scores.values())
for name in models_dict.keys():
    weight = (1.0 / model_mae_scores[name]) / sum_inverse_mae
    initial_weights.append(weight)

print("\nWagi inicjujące dla modeli bazowych:")
for name, weight in zip(models_dict.keys(), initial_weights):
    print(f"{name}: {weight:.4f}")

# Optymalizacja finalnego meta modelu
print("\nOptymalizacja finalnego meta modelu...")
final_study = optuna.create_study(direction="minimize")
final_study.optimize(
    lambda trial: objective_final_meta(trial, oof_predictions, y, initial_weights),
    n_trials=10
)

print(f"\nNajlepsze MAE dla finalnego meta modelu: {final_study.best_value:.4f}")
print("Najlepsze parametry:", final_study.best_params)

# Przygotowanie najlepszego modelu
best_params = final_study.best_params

# Budowa architektury
best_architecture = []
for i in range(best_params["n_layers"]):
    best_architecture.append(best_params[f"units_layer{i}"])

# Standardyzacja dla finalnego modelu
scaler_final = StandardScaler()
oof_predictions_scaled = scaler_final.fit_transform(oof_predictions)

# Trenowanie finalnego modelu
print("\nTrenowanie finalnego meta modelu z najlepszymi parametrami...")
final_meta_model = build_final_meta_model(
    oof_predictions.shape[1],
    best_architecture,
    best_params["dropout_rate"],
    best_params["learning_rate"],
    best_params["weight_decay"],
    initial_weights if best_params["use_weight_layer"] else None,
    best_params["use_weight_layer"],
    best_params["activation"]
)

# Callbacks dla finalnego treningu
early_stop_final = EarlyStopping(
    monitor='loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_final = ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6,
    verbose=1
)

# Trenowanie finalnego modelu
history_final = final_meta_model.fit(
    oof_predictions_scaled, y,
    epochs=200,
    batch_size=best_params["batch_size"],
    verbose=1,
    callbacks=[early_stop_final, reduce_lr_final]
)

# Wizualizacja procesu uczenia
plt.figure(figsize=(12, 6))
plt.plot(history_final.history['loss'], label='Train Loss (MAE)')

if 'val_loss' in history_final.history:
    plt.plot(history_final.history['val_loss'], label='Validation Loss (MAE)')

plt.title('Proces uczenia finalnego meta modelu')
plt.xlabel('Epoka')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

if is_test_iteration:
    # Ocena finalnego modelu
    preds_xgb = xgb_model.predict(X_valid_end)
    preds_cat = cat_model.predict(X_valid_end)
    preds_lgb = lgb_model.predict(X_valid_end)
    preds_ada = ada_pipeline.predict(X_valid_end)
    preds_hgbm = hgbm_model.predict(X_valid_end)

    meta_features_new = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))
    meta_features_new_scaled = scaler.transform(meta_features_new)

    # Predykcje modeli meta dla zbioru testowego
    base_model_preds = []

    # Generowanie predykcji z wszystkich modeli meta
    for name, model in models_dict.items():
        if name == "NN":
            preds = model.predict(meta_features_new_scaled, verbose=0).flatten()
        elif name == "MLP":
            preds = model.predict(meta_features_new_scaled)
        else:
            preds = model.predict(meta_features_new)

        base_model_preds.append(preds)
        print(f"MAE {name} na zbiorze testowym: {mean_absolute_error(y_valid_end, preds):.4f}")

    # Tworzenie macierzy cech dla finalnego meta modelu
    final_meta_features = np.column_stack(base_model_preds)
    final_meta_features_scaled = scaler_final.transform(final_meta_features)

    # Predykcja finalnego meta modelu
    y_pred_final = final_meta_model.predict(final_meta_features_scaled)
    final_mae = mean_absolute_error(y_valid_end, y_pred_final)
    print(f"\nFinalny MAE meta modelu: {final_mae:.4f}")

    # Porównanie predykcji finalnego modelu z wartościami rzeczywistymi
    plt.figure(figsize=(10, 6))
    plt.scatter(y_valid_end, y_pred_final, alpha=0.6, color='blue')
    min_val, max_val = min(y.min(), y_pred_final.min()), max(y.max(), y_pred_final.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    plt.title('Porównanie predykcji finalnego meta modelu z wartościami rzeczywistymi')
    plt.xlabel('Wartości rzeczywiste')
    plt.ylabel('Predykcje')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Analiza ważności modeli bazowych
    if best_params["use_weight_layer"]:
        model_weights = np.array(initial_weights)

        # Wizualizacja wag modeli
        plt.figure(figsize=(12, 6))
        sorted_indices = np.argsort(model_weights)[::-1]  # Sortowanie malejąco
        plt.bar(
            range(len(model_names)),
            [model_weights[i] for i in sorted_indices],
            color='lightgreen'
        )
        plt.xticks(
            range(len(model_names)),
            [model_names[i] for i in sorted_indices],
            rotation=45
        )
        plt.title('Wagi modeli bazowych w finalnym meta modelu')
        plt.ylabel('Waga')
        plt.tight_layout()
        plt.show()



In [ ]:
class EarlyStopping:
    """
    Early stopping dla zatrzymania treningu, gdy model przestaje się poprawiać.
    """
    def __init__(self, patience=7, delta=0, verbose=False):
        """
        Args:
            patience (int): Ile epok czekać zanim zatrzymamy trening po braku poprawy
            delta (float): Minimalna zmiana, aby uznać za poprawę
            verbose (bool): Czy wyświetlać komunikaty
        """
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Wywołanie klasy jako funkcji.

        Args:
            val_loss (float): Wartość walidacyjnej straty
            model (nn.Module): Model do zapisania
        """
        score = -val_loss

        if self.best_score is None:
            # Pierwszy epoka
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            # Brak poprawy
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            # Jest poprawa
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """
        Zapisuje model, gdy walidacyjna strata spada.
        """
        if self.verbose:
            print(f'Zapisywanie modelu (strata spadła z {self.val_loss_min:.6f} do {val_loss:.6f})')
        # Zapisz stan modelu
        self.best_model_state = {k: v.cpu().detach() for k, v in model.state_dict().items()}
        self.val_loss_min = val_loss

    def load_best_model(self, model):
        """
        Ładuje najlepszy model.
        """
        if self.best_model_state is not None:
            model.load_state_dict(self.best_model_state)


def preprocess_original_features(X_orig, categorical_prefix='cat_'):
    """
    Przetwarza oryginalne cechy, przygotowując je do użycia w modelu GNN.

    Args:
        X_orig: DataFrame z oryginalnymi cechami
        categorical_prefix: Prefiks używany dla zmiennych kategorycznych

    Returns:
        tuple: (preprocessed_features, preprocessor, cat_dims, num_dims)
            - preprocessed_features: przetworzone cechy
            - preprocessor: obiekt do przetwarzania nowych danych
            - cat_dims: wymiary cech kategorycznych po przetworzeniu
            - num_dims: liczba cech numerycznych
    """
    # Identyfikacja kolumn kategorycznych i numerycznych
    categorical_cols = [col for col in X_orig.columns if col.startswith(categorical_prefix)]
    numerical_cols = [col for col in X_orig.columns if col not in categorical_cols]

    # Tworzenie przetwarzaczy
    numerical_transformer = Pipeline(steps=[
        ('scaler', RobustScaler()),
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Łączenie przetwarzaczy
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Dopasowanie i transformacja
    preprocessed_features = preprocessor.fit_transform(X_orig)

    # Zapisanie wymiarów kategorycznych
    cat_dims = 0
    if categorical_cols:
        cat_dims = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out().shape[0]

    num_dims = len(numerical_cols)

    return preprocessed_features, preprocessor, cat_dims, num_dims


class EnhancedGNNMetaModel(nn.Module):
    """
    Rozszerzony model GNN, który wykorzystuje zarówno predykcje modeli bazowych
    jak i oryginalne cechy.
    """

    def __init__(self, pred_dim=1, feat_dim=0, hidden_channels=64,
                 num_layers=2, dropout=0.1, initial_weights=None):
        super(EnhancedGNNMetaModel, self).__init__()

        self.pred_dim = pred_dim  # liczba modeli
        self.feat_dim = feat_dim  # liczba oryginalnych cech
        self.num_layers = num_layers

        # Warstwa wejściowa dla predykcji modeli
        self.pred_transform = nn.Linear(1, hidden_channels)

        # Warstwa dla oryginalnych cech
        if feat_dim > 0:
            self.feat_encoder = nn.Sequential(
                nn.Linear(feat_dim, hidden_channels),
                nn.BatchNorm1d(hidden_channels),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        # Warstwy konwolucji grafowej
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(nn.BatchNorm1d(hidden_channels))

        # Warstwa integrująca cechy z grafu i oryginalne
        self.integration_layer = nn.Sequential(
            nn.Linear(hidden_channels * (2 if feat_dim > 0 else 1), hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # Warstwy MLP końcowej predykcji
        self.mlp = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels // 2, 1)
        )

        # Warstwa wag dla modeli
        if initial_weights is not None:
            self.register_buffer('model_weights', torch.FloatTensor(initial_weights).view(1, -1))
        else:
            self.model_weights = None

    def forward(self, x, edge_index, batch=None, orig_features=None):
        """
        Forward pass sieci.

        Args:
            x: Tensor zawierający predykcje modeli [num_nodes, 1]
            edge_index: Indeksy krawędzi grafu [2, num_edges]
            batch: Wskaźniki batch do agregacji grafu
            orig_features: Oryginalne cechy [batch_size, feat_dim]

        Returns:
            Tensor z ostateczną predykcją [batch_size, 1]
        """
        batch_size = x.size(0) // self.pred_dim if self.pred_dim > 0 else x.size(0)

        # Zastosowanie wag modeli
        if self.model_weights is not None and self.pred_dim > 0:
            reshaped_x = x.view(batch_size, self.pred_dim)
            weighted_x = reshaped_x * self.model_weights
            x = weighted_x.view(-1, 1)

        # Przetwarzanie predykcji modeli przez GNN
        x = self.pred_transform(x)
        x = F.relu(x)

        # Warstwy konwolucji grafowej
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.1, training=self.training)

        # Agregacja grafu
        if batch is None:
            batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)

        graph_embedding = global_mean_pool(x, batch)

        # Integracja z oryginalnymi cechami
        if orig_features is not None and self.feat_dim > 0:
            feat_embedding = self.feat_encoder(orig_features)
            combined = torch.cat([graph_embedding, feat_embedding], dim=1)
            combined = self.integration_layer(combined)
        else:
            combined = graph_embedding

        # Końcowa predykcja
        output = self.mlp(combined)

        return output


def create_enhanced_graph_dataset(model_predictions, original_features=None, y=None):
    """
    Tworzy dataset grafów z predykcji modeli i oryginalnych cech.

    Args:
        model_predictions: Predykcje modeli [n_samples, n_models]
        original_features: Przetworzone oryginalne cechy [n_samples, n_features]
        y: Wartości docelowe (opcjonalne) [n_samples]

    Returns:
        list: Lista obiektów Data
    """
    data_list = []
    n_models = model_predictions.shape[1]

    # Tworzenie pełnego grafu - każdy model połączony z każdym innym
    edge_index = []
    for i in range(n_models):
        for j in range(n_models):
            if i != j:
                edge_index.append([i, j])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    for i in range(model_predictions.shape[0]):
        # Predykcje modeli jako cechy węzłów
        x = torch.tensor(model_predictions[i:i + 1, :].reshape(-1, 1), dtype=torch.float)

        # Oryginalne cechy jako atrybut grafu
        graph_data = Data(x=x, edge_index=edge_index)

        if original_features is not None:
            graph_data.orig_features = torch.tensor(
                original_features[i:i + 1, :], dtype=torch.float
            )

        if y is not None:
            graph_data.y = torch.tensor(
                [y.iloc[i] if hasattr(y, 'iloc') else y[i]], dtype=torch.float
            )

        data_list.append(graph_data)

    return data_list


class EnhancedDataLoader(DataLoader):
    """
    Rozszerzony DataLoader, który obsługuje zarówno predykcje modeli, jak i oryginalne cechy.
    """

    def __init__(self, dataset, **kwargs):
        super(EnhancedDataLoader, self).__init__(dataset, **kwargs)

        # Sprawdzenie, czy dataset zawiera oryginalne cechy
        self.has_orig_features = hasattr(dataset[0], 'orig_features')

    def collate_fn(self, data_list):
        batch = super().collate_fn(data_list)

        # Dodanie oryginalnych cech do batcha
        if self.has_orig_features:
            batch.orig_features = torch.cat([data.orig_features for data in data_list], dim=0)

        return batch



def train_enhanced_gnn_epoch(model, data_loader, optimizer, device):
    """
    Trenuje rozszerzony model GNN przez jedną epokę.
    """
    model.train()
    total_loss = 0

    for batch in data_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass z oryginalnymi cechami
        if hasattr(batch, 'orig_features'):
            pred = model(batch.x, batch.edge_index, batch.batch, batch.orig_features)
        else:
            pred = model(batch.x, batch.edge_index, batch.batch)

        # Obliczenie straty
        target = batch.y.view(-1, 1)
        loss = F.l1_loss(pred, target)  # MAE loss

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_graphs

    return total_loss / len(data_loader.dataset)


def validate_enhanced_gnn(model, data_loader, device):
    """
    Waliduje rozszerzony model GNN.
    """
    model.eval()
    predictions = []
    targets = []

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)

            # Forward pass z oryginalnymi cechami
            if hasattr(batch, 'orig_features'):
                pred = model(batch.x, batch.edge_index, batch.batch, batch.orig_features)
            else:
                pred = model(batch.x, batch.edge_index, batch.batch)

            predictions.append(pred.cpu().numpy())
            targets.append(batch.y.view(-1, 1).cpu().numpy())

    predictions = np.vstack(predictions)
    targets = np.vstack(targets)

    mae = mean_absolute_error(targets, predictions)
    return mae



def objective_enhanced_gnn_meta(trial, model_predictions, orig_features, y, initial_weights=None):
    """
    Funkcja celu dla optymalizacji rozszerzonego modelu GNN.
    """
    # Hiperparametry do optymalizacji
    hidden_channels = trial.suggest_int("hidden_channels", 100, 500)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.01, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    use_weight_layer = trial.suggest_categorical("use_weight_layer", [True, False])

    # Walidacja krzyżowa
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    mae_scores = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for train_idx, val_idx in kf.split(model_predictions):
        # Podział danych
        X_train_pred, X_val_pred = model_predictions[train_idx], model_predictions[val_idx]
        X_train_orig, X_val_orig = orig_features[train_idx], orig_features[val_idx]
        y_train, y_val = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx], y.iloc[val_idx] if hasattr(y, 'iloc') else y[val_idx]

        # Standardyzacja predykcji modeli
        scaler_pred = RobustScaler()
        X_train_pred_scaled = scaler_pred.fit_transform(X_train_pred)
        X_val_pred_scaled = scaler_pred.transform(X_val_pred)

        # Przygotowanie datasetów
        weights_to_use = initial_weights if use_weight_layer else None
        train_dataset = create_enhanced_graph_dataset(X_train_pred_scaled, X_train_orig, y_train)
        val_dataset = create_enhanced_graph_dataset(X_val_pred_scaled, X_val_orig, y_val)

        # Utworzenie loaderów
        train_loader = EnhancedDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = EnhancedDataLoader(val_dataset, batch_size=batch_size)

        # Budowa modelu
        model = EnhancedGNNMetaModel(
            pred_dim=model_predictions.shape[1],
            feat_dim=orig_features.shape[1] if orig_features is not None else 0,
            hidden_channels=hidden_channels,
            num_layers=num_layers,
            dropout=dropout,
            initial_weights=weights_to_use
        ).to(device)

        # Optymalizator
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay
        )

        # Scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6
        )

        # Early stopping
        early_stopping = EarlyStopping(patience=15)

        # Trenowanie
        for epoch in range(100):
            train_loss = train_enhanced_gnn_epoch(model, train_loader, optimizer, device)
            val_mae = validate_enhanced_gnn(model, val_loader, device)

            scheduler.step(val_mae)
            early_stopping(val_mae, model)

            if early_stopping.early_stop:
                break

        # Załadowanie najlepszego modelu
        early_stopping.load_best_model(model)

        # Ocena finalnego modelu
        final_mae = validate_enhanced_gnn(model, val_loader, device)
        mae_scores.append(final_mae)

    mean_mae = np.mean(mae_scores)
    print(f"Enhanced GNN: Warstwy={num_layers}, Units={hidden_channels}, MAE={mean_mae:.4f}")
    return mean_mae


def train_final_enhanced_gnn_model(model_predictions, orig_features, y, best_params,
                                   initial_weights=None, test_pred=None, test_orig=None, y_test=None):
    """
    Trenuje finalny rozszerzony model GNN.
    """
    # Standardyzacja predykcji modeli
    scaler_pred = RobustScaler()
    X_pred_scaled = scaler_pred.fit_transform(model_predictions)

    # Przygotowanie danych treningowych
    train_dataset = create_enhanced_graph_dataset(X_pred_scaled, orig_features, y)
    train_loader = EnhancedDataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    # Przygotowanie danych testowych
    test_loader = None
    if test_pred is not None and y_test is not None:
        test_pred_scaled = scaler_pred.transform(test_pred)
        test_dataset = create_enhanced_graph_dataset(test_pred_scaled, test_orig, y_test)
        test_loader = EnhancedDataLoader(test_dataset, batch_size=best_params['batch_size'])

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Trenowanie na: {device}")

    # Budowa modelu
    weights_to_use = initial_weights if best_params['use_weight_layer'] else None
    model = EnhancedGNNMetaModel(
        pred_dim=model_predictions.shape[1],
        feat_dim=orig_features.shape[1] if orig_features is not None else 0,
        hidden_channels=best_params['hidden_channels'],
        num_layers=best_params['num_layers'],
        dropout=best_params['dropout'],
        initial_weights=weights_to_use
    ).to(device)

    # Optymalizator
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-6
    )

    # Early stopping
    early_stopping = EarlyStopping(patience=20)

    # Historia treningu
    history = {
        'train_loss': [],
        'test_mae': [] if test_loader is not None else None
    }

    # Trenowanie modelu
    print("\nRozpoczęcie treningu finalnego rozszerzonego modelu GNN...")
    for epoch in range(300):
        # Trening
        train_loss = train_enhanced_gnn_epoch(model, train_loader, optimizer, device)
        history['train_loss'].append(train_loss)

        # Walidacja
        if test_loader is not None:
            test_mae = validate_enhanced_gnn(model, test_loader, device)
            history['test_mae'].append(test_mae)
            print(f"Epoka {epoch + 1}/300, Strata: {train_loss:.4f}, Test MAE: {test_mae:.4f}")
            scheduler.step(test_mae)
            early_stopping(test_mae, model)
        else:
            print(f"Epoka {epoch + 1}/300, Strata: {train_loss:.4f}")
            scheduler.step(train_loss)
            early_stopping(train_loss, model)

        if early_stopping.early_stop:
            print(f"Early stopping w epoce {epoch + 1}")
            break

    # Załadowanie najlepszego modelu
    early_stopping.load_best_model(model)

    # Ocena końcowa
    test_mae = None
    if test_loader is not None:
        test_mae = validate_enhanced_gnn(model, test_loader, device)
        print(f"\nKońcowy MAE na zbiorze testowym: {test_mae:.4f}")

    return model, history, test_mae, scaler_pred


def predict_with_enhanced_gnn(model, model_predictions, orig_features, scaler_pred, device=None):
    """
    Generuje predykcje przy użyciu rozszerzonego modelu GNN.
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.eval()
    model = model.to(device)

    # Standaryzacja predykcji modeli
    pred_scaled = scaler_pred.transform(model_predictions)

    # Przygotowanie danych
    dataset = create_enhanced_graph_dataset(pred_scaled, orig_features)
    loader = EnhancedDataLoader(dataset, batch_size=64)

    # Generowanie predykcji
    predictions = []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)

            # Forward pass
            if hasattr(batch, 'orig_features'):
                pred = model(batch.x, batch.edge_index, batch.batch, batch.orig_features)
            else:
                pred = model(batch.x, batch.edge_index, batch.batch)

            predictions.append(pred.cpu().numpy())

    return np.vstack(predictions)



# Przetwarzanie oryginalnych cech
print("Przygotowanie oryginalnych cech...")
X_orig_processed, orig_preprocessor, cat_dims, num_dims = preprocess_original_features(X)
print(
    f"Przetworzono cechy: {X_orig_processed.shape[1]} cech łącznie (numeryczne: {num_dims}, kategoryczne: {cat_dims})")

# Pozyskanie oryginalnych cech dla zbioru testowego
if is_test_iteration and 'X_valid_end' in globals():
    X_valid_orig_processed = orig_preprocessor.transform(X_valid_end)
else:
    X_valid_orig_processed = None

# Optymalizacja rozszerzonego modelu GNN
print("\nOptymalizacja finalnego rozszerzonego meta modelu GNN...")
enhanced_gnn_study = optuna.create_study(direction="minimize")
enhanced_gnn_study.optimize(
    lambda trial: objective_enhanced_gnn_meta(
        trial,
        oof_predictions,
        X_orig_processed,
        y,
        initial_weights
    ),
    n_trials=5
)

print(f"\nNajlepsze MAE dla rozszerzonego modelu GNN: {enhanced_gnn_study.best_value:.4f}")
print("Najlepsze parametry:", enhanced_gnn_study.best_params)

# Trenowanie finalnego rozszerzonego modelu GNN
final_enhanced_model, history_enhanced, final_enhanced_mae, scaler_pred = train_final_enhanced_gnn_model(
    oof_predictions,
    X_orig_processed,
    y,
    enhanced_gnn_study.best_params,
    initial_weights if enhanced_gnn_study.best_params['use_weight_layer'] else None,
    final_meta_features if is_test_iteration else None,
    X_valid_orig_processed if is_test_iteration else None,
    y_valid_end if is_test_iteration else None
)

# Wizualizacja procesu uczenia
plt.figure(figsize=(12, 6))
plt.plot(history_enhanced['train_loss'], label='Train Loss')

if history_enhanced['test_mae'] is not None:
    plt.plot(history_enhanced['test_mae'], label='Test MAE')

plt.title('Proces uczenia rozszerzonego meta modelu GNN')
plt.xlabel('Epoka')
plt.ylabel('Wartość')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Jeśli testowa iteracja, generujemy predykcje
if is_test_iteration:
    y_pred_enhanced = predict_with_enhanced_gnn(
        final_enhanced_model,
        final_meta_features,
        X_valid_orig_processed,
        scaler_pred
    ).flatten()

    final_enhanced_mae = mean_absolute_error(y_valid_end, y_pred_enhanced)
    print(f"\nFinalny MAE rozszerzonego meta modelu GNN: {final_enhanced_mae:.4f}")

    # Porównanie predykcji z wartościami rzeczywistymi
    plt.figure(figsize=(10, 6))
    plt.scatter(y_valid_end, y_pred_enhanced, alpha=0.6, color='purple')
    min_val = min(y.min(), y_pred_enhanced.min())
    max_val = max(y.max(), y_pred_enhanced.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    plt.title('Porównanie predykcji rozszerzonego meta modelu GNN')
    plt.xlabel('Wartości rzeczywiste')
    plt.ylabel('Predykcje')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Porównanie wszystkich modeli
    models_comparison = {
        'Najlepszy model bazowy': min(model_mae_scores.values()),
        'Standardowy meta model': final_mae,
        'Rozszerzony GNN meta model': final_enhanced_mae
    }

    plt.figure(figsize=(12, 6))
    plt.bar(
        range(len(models_comparison)),
        list(models_comparison.values()),
        color=['skyblue', 'blue', 'green', 'purple']
    )
    plt.xticks(range(len(models_comparison)), list(models_comparison.keys()), rotation=45)
    plt.title('Porównanie MAE dla różnych podejść')
    plt.ylabel('MAE')
    plt.grid(True, linestyle='--', alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()




In [ ]:
def predict_with_nn_model(model, features, batch_size=32, device=None):
    """
    Generuje predykcje z modelu sieci neuronowej TensorFlow.

    Args:
        model: Model TensorFlow/Keras
        features: Cechy wejściowe do predykcji [n_samples, n_features]
        batch_size: Rozmiar batcha dla predykcji
        device: Parametr ignorowany (dla spójności interfejsu z funkcją GNN)

    Returns:
        numpy.ndarray: Predykcje modelu sieci neuronowej
    """
    # Sprawdzenie czy dane są już przeskalowane
    if isinstance(features, np.ndarray) and features.ndim == 2:
        X_features = features
    else:
        # Konwersja danych
        X_features = np.array(features)

    # Wykonanie predykcji
    predictions = model.predict(X_features, batch_size=batch_size, verbose=0)

    # Upewnienie się, że wyjście ma odpowiedni format
    if predictions.ndim > 1 and predictions.shape[1] == 1:
        predictions = predictions.flatten()

    return predictions


def predict_with_gnn_model(model, features, batch_size=64, device=None):
    """
    Generuje predykcje z modelu grafowej sieci neuronowej PyTorch.

    Args:
        model: Model PyTorch GNN
        features: Cechy wejściowe do predykcji [n_samples, n_features]
        batch_size: Rozmiar batcha dla predykcji
        device: Urządzenie do obliczeń (CPU/GPU)

    Returns:
        numpy.ndarray: Predykcje modelu GNN
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.eval()
    model = model.to(device)

    if isinstance(model, EnhancedGNNMetaModel):
        n_samples = features.shape[0]
        n_models = model.pred_dim if hasattr(model, 'pred_dim') and model.pred_dim > 0 else 2

        # Tworzenie sztucznych predykcji modeli, potrzebnych do struktury grafu
        dummy_predictions = np.zeros((n_samples, n_models))

        # Standaryzacja sztucznych predykcji
        scaler = RobustScaler()
        dummy_scaled = scaler.fit_transform(dummy_predictions)

        # Tworzenie datasetu grafowego
        dataset = create_enhanced_graph_dataset(dummy_scaled, features)
        loader = EnhancedDataLoader(dataset, batch_size=batch_size)

        # Generowanie predykcji
        predictions = []

        with torch.no_grad():
            for batch in loader:
                batch = batch.to(device)

                # Forward pass
                if hasattr(batch, 'orig_features'):
                    pred = model(batch.x, batch.edge_index, batch.batch, batch.orig_features)
                else:
                    pred = model(batch.x, batch.edge_index, batch.batch)

                predictions.append(pred.cpu().numpy())

        return np.vstack(predictions).flatten()
    else:
        # Dla innych modeli PyTorch, które nie są GNN
        # Konwersja danych wejściowych na tensor
        X_tensor = torch.tensor(features, dtype=torch.float32).to(device)

        # Podział na batche dla efektywności
        dataset = torch.utils.data.TensorDataset(X_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

        # Predykcja
        predictions = []

        with torch.no_grad():
            for batch in loader:
                batch_x = batch[0]
                outputs = model(batch_x)
                predictions.append(outputs.cpu().numpy())

        return np.vstack(predictions).flatten()

class AttentionFusionModel(nn.Module):
    """
    Model łączący prognozy z kilku źródeł (standardowa sieć neuronowa, GNN,
    oryginalne cechy) przy użyciu mechanizmu uwagi (attention).
    """

    def __init__(self, nn_dim=1, gnn_dim=1, orig_feat_dim=0,
                 hidden_size=64, dropout=0.1):
        super(AttentionFusionModel, self).__init__()

        self.nn_dim = nn_dim
        self.gnn_dim = gnn_dim
        self.orig_feat_dim = orig_feat_dim

        # Liczba źródeł (modeli) do łączenia
        self.num_sources = sum([
            1 if nn_dim > 0 else 0,  # Standardowa sieć neuronowa
            1 if gnn_dim > 0 else 0,  # Grafowa sieć neuronowa
            1 if orig_feat_dim > 0 else 0  # Oryginalne cechy
        ])

        # Warstwy enkodujące dla każdego źródła danych
        if nn_dim > 0:
            self.nn_encoder = nn.Sequential(
                nn.Linear(nn_dim, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        if gnn_dim > 0:
            self.gnn_encoder = nn.Sequential(
                nn.Linear(gnn_dim, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        if orig_feat_dim > 0:
            self.feat_encoder = nn.Sequential(
                nn.Linear(orig_feat_dim, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        # Mechanizm uwagi (attention)
        # Query, Key, Value transformacje
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)

        # Skalowanie
        self.scale = math.sqrt(hidden_size)

        # Finalna warstwa predykcji
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, nn_pred=None, gnn_pred=None, orig_features=None):
        """
        Forward pass modelu uwagi.

        Args:
            nn_pred: Predykcje ze standardowej sieci neuronowej [batch_size, nn_dim]
            gnn_pred: Predykcje z GNN [batch_size, gnn_dim]
            orig_features: Oryginalne cechy [batch_size, orig_feat_dim]

        Returns:
            Tensor: Finalna predykcja z połączonych źródeł [batch_size, 1]
        """
        # Lista zakodowanych reprezentacji z każdego źródła
        encoded_sources = []
        batch_size = None

        # Enkodowanie predykcji z sieci neuronowej
        if nn_pred is not None and self.nn_dim > 0:
            batch_size = nn_pred.size(0)
            nn_encoded = self.nn_encoder(nn_pred)
            encoded_sources.append(nn_encoded)

        # Enkodowanie predykcji z GNN
        if gnn_pred is not None and self.gnn_dim > 0:
            batch_size = gnn_pred.size(0)
            gnn_encoded = self.gnn_encoder(gnn_pred)
            encoded_sources.append(gnn_encoded)

        # Enkodowanie oryginalnych cech
        if orig_features is not None and self.orig_feat_dim > 0:
            batch_size = orig_features.size(0)
            feat_encoded = self.feat_encoder(orig_features)
            encoded_sources.append(feat_encoded)

        if not encoded_sources:
            raise ValueError("At least one source of predictions must be provided")

        # Konwersja listy na tensor [batch_size, num_sources, hidden_size]
        encoded_stack = torch.stack(encoded_sources, dim=1)

        # Mechanizm uwagi (attention)
        # Transformacja Query, Key, Value
        queries = self.query(encoded_stack)  # [batch_size, num_sources, hidden_size]
        keys = self.key(encoded_stack)  # [batch_size, num_sources, hidden_size]
        values = self.value(encoded_stack)  # [batch_size, num_sources, hidden_size]

        # Obliczenie wag uwagi
        # [batch_size, num_sources, num_sources]
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / self.scale
        attention_weights = F.softmax(attention_scores, dim=-1)

        # Ważone zsumowanie wartości
        # [batch_size, num_sources, hidden_size]
        context_vector = torch.matmul(attention_weights, values)

        # Średnia po źródłach
        # [batch_size, hidden_size]
        context_vector = context_vector.mean(dim=1)

        # Finalna predykcja
        output = self.output_layer(context_vector)

        return output

    def get_attention_weights(self, nn_pred=None, gnn_pred=None, orig_features=None):
        """
        Metoda do pobrania wag uwagi dla wizualizacji i interpretacji.

        Returns:
            numpy.ndarray: Macierz wag uwagi [batch_size, num_sources, num_sources]
        """
        encoded_sources = []

        # Enkodowanie predykcji z sieci neuronowej
        if nn_pred is not None and self.nn_dim > 0:
            nn_encoded = self.nn_encoder(nn_pred)
            encoded_sources.append(nn_encoded)

        # Enkodowanie predykcji z GNN
        if gnn_pred is not None and self.gnn_dim > 0:
            gnn_encoded = self.gnn_encoder(gnn_pred)
            encoded_sources.append(gnn_encoded)

        # Enkodowanie oryginalnych cech
        if orig_features is not None and self.orig_feat_dim > 0:
            feat_encoded = self.feat_encoder(orig_features)
            encoded_sources.append(feat_encoded)

        # Konwersja listy na tensor [batch_size, num_sources, hidden_size]
        encoded_stack = torch.stack(encoded_sources, dim=1)

        # Obliczenie wag uwagi
        queries = self.query(encoded_stack)
        keys = self.key(encoded_stack)
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / self.scale
        attention_weights = F.softmax(attention_scores, dim=-1)

        return attention_weights.detach().cpu().numpy()


def prepare_fusion_data(nn_predictions, gnn_predictions, orig_features=None):
    """
    Przygotowuje dane do modelu uwagi, zapewniając spójność tensorów.

    Args:
        nn_predictions: Predykcje z sieci neuronowej
        gnn_predictions: Predykcje z GNN
        orig_features: Oryginalne cechy (opcjonalnie)

    Returns:
        tuple: (nn_tensor, gnn_tensor, orig_tensor)
    """
    # Konwersja na tensory PyTorch
    nn_tensor = torch.tensor(nn_predictions, dtype=torch.float32)
    gnn_tensor = torch.tensor(gnn_predictions, dtype=torch.float32)

    # Sprawdzenie właściwego kształtu [batch_size, feature_dim]
    if nn_tensor.dim() == 1:
        nn_tensor = nn_tensor.unsqueeze(1)
    if gnn_tensor.dim() == 1:
        gnn_tensor = gnn_tensor.unsqueeze(1)

    # Oryginalne cechy
    orig_tensor = None
    if orig_features is not None:
        if isinstance(orig_features, np.ndarray):
            orig_tensor = torch.tensor(orig_features, dtype=torch.float32)
        else:
            orig_tensor = orig_features

    return nn_tensor, gnn_tensor, orig_tensor

def visualize_attention_weights(model, nn_pred, gnn_pred, orig_features=None, sample_idx=None):
    """
    Wizualizuje wagi uwagi dla wybranych próbek.

    Args:
        model: Wytrenowany model uwagi
        nn_pred: Predykcje z sieci neuronowej
        gnn_pred: Predykcje z GNN
        orig_features: Oryginalne cechy (opcjonalnie)
        sample_idx: Indeksy próbek do wizualizacji (None = wszystkie)
    """
    # Przygotowanie danych
    nn_tensor, gnn_tensor, orig_tensor = prepare_fusion_data(
        nn_pred, gnn_pred, orig_features
    )

    # Wybór próbek
    if sample_idx is not None:
        nn_tensor = nn_tensor[sample_idx]
        gnn_tensor = gnn_tensor[sample_idx]
        if orig_tensor is not None:
            orig_tensor = orig_tensor[sample_idx]

    # Pobranie wag uwagi
    model.eval()
    with torch.no_grad():
        attention_weights = model.get_attention_weights(nn_tensor, gnn_tensor, orig_tensor)

    # Nazwy źródeł do wizualizacji
    source_names = []
    if model.nn_dim > 0:
        source_names.append("Sieć neuronowa")
    if model.gnn_dim > 0:
        source_names.append("Grafowa sieć")
    if model.orig_feat_dim > 0:
        source_names.append("Oryginalne cechy")

    # Wizualizacja
    num_samples = attention_weights.shape[0]
    max_samples = min(5, num_samples)

    plt.figure(figsize=(15, 4 * max_samples))

    for i in range(max_samples):
        plt.subplot(max_samples, 1, i + 1)
        sns.heatmap(
            attention_weights[i],
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            xticklabels=source_names,
            yticklabels=source_names
        )
        plt.title(f"Uwaga dla próbki {i + 1}")

    plt.tight_layout()
    plt.show()

    # Średnie wagi uwagi dla wszystkich próbek
    mean_weights = attention_weights.mean(axis=0)

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        mean_weights,
        annot=True,
        fmt=".2f",
        cmap="YlGnBu",
        xticklabels=source_names,
        yticklabels=source_names
    )
    plt.title("Średnie wagi uwagi dla wszystkich próbek")
    plt.tight_layout()
    plt.show()

def train_final_attention_fusion_model(model_predictions, orig_features, y,
                                       nn_model=None, gnn_model=None,
                                       hidden_size=64, dropout=0.2, lr=0.001,
                                       batch_size=32, epochs=100, validation_ratio=0.1,
                                       patience=15, device=None, is_test=False,
                                       test_predictions=None, test_features=None, test_y=None):
    """
    Trenuje finalny model uwagi (Attention Fusion) na całym zbiorze danych lub z walidacją.

    Args:
        model_predictions: Predykcje modeli bazowych na zbiorze treningowym [n_samples, n_models]
        orig_features: Oryginalne cechy zbioru treningowego [n_samples, n_features]
        y: Wartości docelowe zbioru treningowego [n_samples]
        nn_model: Model standardowej sieci neuronowej (opcjonalnie)
        gnn_model: Model grafowej sieci neuronowej (opcjonalnie)
        hidden_size: Rozmiar warstw ukrytych
        dropout: Współczynnik dropout
        lr: Learning rate
        batch_size: Rozmiar batcha
        epochs: Maksymalna liczba epok
        validation_ratio: Część danych do walidacji (używane tylko gdy is_test=False)
        patience: Liczba epok do early stopping
        device: Urządzenie do treningu (CPU/GPU)
        is_test: Czy mamy dostępny zewnętrzny zbiór testowy
        test_predictions: Predykcje modeli bazowych na zbiorze testowym (dla is_test=True)
        test_features: Oryginalne cechy zbioru testowego (dla is_test=True)
        test_y: Wartości docelowe zbioru testowego (dla is_test=True)

    Returns:
        tuple: (model, history, test_mae)
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Trening modelu uwagi na: {device}")

    # Generowanie predykcji z nn_model i gnn_model
    if nn_model is not None and model_predictions is None:
        print("Generowanie predykcji z modelu NN...")
        nn_train_pred = predict_with_nn_model(nn_model, orig_features).reshape(-1, 1)

        if is_test and test_features is not None:
            nn_test_pred = predict_with_nn_model(nn_model, test_features).reshape(-1, 1)
    else:
        nn_train_pred = model_predictions[:, 0].reshape(-1, 1)
        nn_test_pred = test_predictions[:, 0].reshape(-1, 1) if is_test and test_predictions is not None else None

    if gnn_model is not None and model_predictions is None:
        print("Generowanie predykcji z modelu GNN...")
        gnn_train_pred = predict_with_gnn_model(gnn_model, orig_features).reshape(-1, 1)

        if is_test and test_features is not None:
            gnn_test_pred = predict_with_gnn_model(gnn_model, test_features).reshape(-1, 1)
    else:
        gnn_train_pred = model_predictions[:, 1].reshape(-1, 1) if model_predictions.shape[1] > 1 else nn_train_pred
        gnn_test_pred = test_predictions[:, 1].reshape(-1, 1) if is_test and test_predictions is not None and test_predictions.shape[1] > 1 else nn_test_pred

    # Konwersja wartości docelowych do numpy
    y_np = y.values if isinstance(y, pd.Series) or isinstance(y, pd.DataFrame) else y
    y_tensor = torch.tensor(y_np, dtype=torch.float32).reshape(-1, 1)

    # Przygotowanie danych treningowych
    nn_train_tensor = torch.tensor(nn_train_pred, dtype=torch.float32)
    gnn_train_tensor = torch.tensor(gnn_train_pred, dtype=torch.float32)
    orig_train_tensor = torch.tensor(orig_features, dtype=torch.float32)

    # Przygotowanie danych testowych
    nn_test_tensor = torch.tensor(nn_test_pred, dtype=torch.float32) if is_test and nn_test_pred is not None else None
    gnn_test_tensor = torch.tensor(gnn_test_pred, dtype=torch.float32) if is_test and gnn_test_pred is not None else None
    orig_test_tensor = torch.tensor(test_features, dtype=torch.float32) if is_test and test_features is not None else None

    if is_test and test_y is not None:
        test_y_np = test_y.values if isinstance(test_y, pd.Series) or isinstance(test_y, pd.DataFrame) else test_y
        y_test_tensor = torch.tensor(test_y_np, dtype=torch.float32).reshape(-1, 1)
    else:
        y_test_tensor = None

    if is_test and all(x is not None for x in [nn_test_tensor, gnn_test_tensor, orig_test_tensor, y_test_tensor]):
        # Używamy całego zbioru treningowego + zewnętrzny zbiór testowy
        train_indices = list(range(len(y_tensor)))
        has_validation = True
    else:
        # Brak zbioru testowego, tworzymy walidację z części zbioru treningowego
        dataset_size = len(y_tensor)
        indices = list(range(dataset_size))

        if validation_ratio > 0:
            split = int(np.floor(validation_ratio * dataset_size))
            np.random.shuffle(indices)
            train_indices, val_indices = indices[split:], indices[:split]
            has_validation = True

            # Tworzymy "quasi-testowy" zbiór z danych walidacyjnych
            nn_test_tensor = nn_train_tensor[val_indices]
            gnn_test_tensor = gnn_train_tensor[val_indices]
            orig_test_tensor = orig_train_tensor[val_indices]
            y_test_tensor = y_tensor[val_indices]
        else:
            # Brak walidacji, używamy całego zbioru
            train_indices = indices
            has_validation = False

    # Inicjalizacja modelu
    model = AttentionFusionModel(
        nn_dim=nn_train_tensor.size(1),
        gnn_dim=gnn_train_tensor.size(1),
        orig_feat_dim=orig_train_tensor.size(1),
        hidden_size=hidden_size,
        dropout=dropout
    ).to(device)

    # Optymizator i funkcja straty
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.L1Loss()  # MAE loss

    # Scheduler i early stopping
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=patience//3, min_lr=1e-6
    )
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    # Historia treningu
    history = {'train_loss': [], 'val_mae': [] if has_validation else None}

    # Przygotowanie DataLoader dla treningu
    train_dataset = TensorDataset(
        nn_train_tensor[train_indices],
        gnn_train_tensor[train_indices],
        orig_train_tensor[train_indices],
        y_tensor[train_indices]
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # DataLoader dla walidacji
    val_loader = None
    if has_validation:
        val_dataset = TensorDataset(
            nn_test_tensor,
            gnn_test_tensor,
            orig_test_tensor,
            y_test_tensor
        )
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Główna pętla treningowa
    print(f"Rozpoczęcie treningu modelu uwagi na {len(train_indices)} próbkach...")
    print(f"Walidacja: {'Tak' if has_validation else 'Nie'}")

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for batch_nn, batch_gnn, batch_orig, batch_y in train_loader:
            # Przesłanie danych
            batch_nn = batch_nn.to(device)
            batch_gnn = batch_gnn.to(device)
            batch_orig = batch_orig.to(device)
            batch_y = batch_y.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_nn, batch_gnn, batch_orig)

            # Obliczenie straty
            loss = criterion(outputs, batch_y)

            # Backward pass
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * batch_y.size(0)

        avg_train_loss = total_train_loss / len(train_dataset)
        history['train_loss'].append(avg_train_loss)

        # Walidacja, jeśli mamy dane walidacyjne
        val_loss = None
        if has_validation and val_loader is not None:
            model.eval()
            total_val_loss = 0

            with torch.no_grad():
                for batch_nn, batch_gnn, batch_orig, batch_y in val_loader:
                    # Przesłanie danych
                    batch_nn = batch_nn.to(device)
                    batch_gnn = batch_gnn.to(device)
                    batch_orig = batch_orig.to(device)
                    batch_y = batch_y.to(device)

                    # Forward pass
                    outputs = model(batch_nn, batch_gnn, batch_orig)

                    # Akumulacja straty
                    val_loss = criterion(outputs, batch_y)
                    total_val_loss += val_loss.item() * batch_y.size(0)

            val_loss = total_val_loss / len(val_dataset)
            history['val_mae'].append(val_loss)

            print(f"Epoka {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val MAE: {val_loss:.4f}")

            # Aktualizacja schedulera i early stopping
            scheduler.step(val_loss)
            early_stopping(val_loss, model)
        else:
            print(f"Epoka {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")
            scheduler.step(avg_train_loss)
            early_stopping(avg_train_loss, model)

        # Sprawdzenie early stopping
        if early_stopping.early_stop:
            print(f"Early stopping w epoce {epoch+1}")
            break

    # Załadowanie najlepszego modelu
    early_stopping.load_best_model(model)

    # Ocena końcowa, jeśli mamy dane walidacyjne
    final_mae = None
    if has_validation and val_loader is not None:
        model.eval()
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for batch_nn, batch_gnn, batch_orig, batch_y in val_loader:
                # Przesłanie danych
                batch_nn = batch_nn.to(device)
                batch_gnn = batch_gnn.to(device)
                batch_orig = batch_orig.to(device)

                # Forward pass
                outputs = model(batch_nn, batch_gnn, batch_orig)

                # Zbieranie predykcji i celów
                all_preds.append(outputs.cpu().numpy())
                all_targets.append(batch_y.cpu().numpy())

        # Obliczenie końcowego MAE
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        final_mae = mean_absolute_error(all_targets, all_preds)
        print(f"Finalny MAE modelu uwagi: {final_mae:.4f}")

    return model, history, final_mae

def generate_fusion_predictions(fusion_model, nn_predictions, gnn_predictions, orig_features, device=None):
    """
    Generuje predykcje z modelu uwagi dla dowolnych danych.

    Args:
        fusion_model: Wytrenowany model uwagi
        nn_predictions: Predykcje z sieci neuronowej
        gnn_predictions: Predykcje z GNN
        orig_features: Oryginalne cechy
        device: Urządzenie do obliczeń

    Returns:
        numpy.ndarray: Predykcje modelu uwagi
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    fusion_model.eval()
    fusion_model = fusion_model.to(device)

    # Przygotowanie danych
    nn_tensor = torch.tensor(nn_predictions, dtype=torch.float32).to(device)
    gnn_tensor = torch.tensor(gnn_predictions, dtype=torch.float32).to(device)
    orig_tensor = torch.tensor(orig_features, dtype=torch.float32).to(device)

    # Zapewnienie odpowiedniego kształtu
    if nn_tensor.dim() == 1:
        nn_tensor = nn_tensor.unsqueeze(1)
    if gnn_tensor.dim() == 1:
        gnn_tensor = gnn_tensor.unsqueeze(1)

    # Tworzenie DataLoadera dla większej efektywności
    dataset = TensorDataset(nn_tensor, gnn_tensor, orig_tensor)
    loader = DataLoader(dataset, batch_size=64)

    # Generowanie predykcji
    all_preds = []

    with torch.no_grad():
        for batch_nn, batch_gnn, batch_orig in loader:
            # Forward pass
            outputs = fusion_model(batch_nn, batch_gnn, batch_orig)
            all_preds.append(outputs.cpu().numpy())

    return np.vstack(all_preds)

import torch
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import math

# Pełny trening modelu uwagi
print("\nTrening finalnego modelu uwagi (Attention Fusion)...")

# Sprawdzenie czy mamy predykcje modeli
have_model_predictions = False
if 'oof_predictions' in globals() and oof_predictions is not None:
    have_model_predictions = True
    nn_train_preds = oof_predictions[:, 0].reshape(-1, 1)
    gnn_train_preds = oof_predictions[:, 1].reshape(-1, 1) if oof_predictions.shape[1] > 1 else nn_train_preds
    print(f"Używanie istniejących predykcji z {oof_predictions.shape[1]} modeli bazowych")

if is_test_iteration:
    print("Trenowanie modelu uwagi z użyciem zewnętrznego zbioru testowego")

    # Sprawdzenie i przygotowanie danych testowych
    nn_test_preds = None
    gnn_test_preds = None

    if 'y_pred_final' in globals() and 'y_pred_enhanced' in globals():
        nn_test_preds = y_pred_final.reshape(-1, 1)
        gnn_test_preds = y_pred_enhanced.reshape(-1, 1)
    elif 'final_meta_features' in globals():
        nn_test_preds = final_meta_features[:, 0].reshape(-1, 1)
        gnn_test_preds = final_meta_features[:, 1].reshape(-1, 1) if final_meta_features.shape[1] > 1 else nn_test_preds

    # Trening modelu uwagi z zewnętrzną walidacją
    fusion_model, fusion_history, fusion_mae = train_final_attention_fusion_model(
        oof_predictions if have_model_predictions else None,
        X_orig_processed,
        y,
        final_meta_model if not have_model_predictions else None,
        final_enhanced_model if not have_model_predictions else None,
        hidden_size=128,
        dropout=0.2,
        lr=0.001,
        batch_size=32,
        epochs=1000,
        validation_ratio=0.1,
        patience=20,
        is_test=True,
        test_predictions=np.hstack([nn_test_preds, gnn_test_preds]) if nn_test_preds is not None else None,
        test_features=X_valid_orig_processed,
        test_y=y_valid_end
    )

    # Wizualizacja procesu uczenia
    plt.figure(figsize=(12, 6))
    plt.plot(fusion_history['train_loss'], label='Train Loss')
    if fusion_history['val_mae'] is not None:
        plt.plot(fusion_history['val_mae'], label='Val MAE')
    plt.title('Proces uczenia finalnego modelu uwagi (Attention Fusion)')
    plt.xlabel('Epoka')
    plt.ylabel('Wartość')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Wizualizacja wag uwagi
    visualize_attention_weights(
        fusion_model,
        nn_test_preds,
        gnn_test_preds,
        X_valid_orig_processed,
        sample_idx=np.random.choice(len(nn_test_preds), size=5, replace=False)
    )

    # Porównanie wszystkich modeli
    models_comparison = {
        'Najlepszy model bazowy': min(model_mae_scores.values()),
        'Standardowy meta model': final_mae,
        'Rozszerzony GNN meta model': final_enhanced_mae,
        'Model uwagi (Attention Fusion)': fusion_mae
    }

    plt.figure(figsize=(12, 6))
    plt.bar(
        range(len(models_comparison)),
        list(models_comparison.values()),
        color=['skyblue', 'blue', 'green', 'red']
    )
    plt.xticks(range(len(models_comparison)), list(models_comparison.keys()), rotation=45)
    plt.title('Porównanie MAE dla różnych podejść')
    plt.ylabel('MAE')
    plt.grid(True, linestyle='--', alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

else:
    # Przypadek treningu finalnego modelu bez zewnętrznego zbioru testowego
    print("Trenowanie finalnego modelu uwagi na pełnym zbiorze z cross-validacją")

    # Trening modelu uwagi z walidacją wewnętrzną
    fusion_model, fusion_history, _ = train_final_attention_fusion_model(
        oof_predictions if have_model_predictions else None,
        X_orig_processed,
        y,
        final_meta_model if not have_model_predictions and 'final_meta_model' in globals() else None,
        final_enhanced_model if not have_model_predictions and 'final_enhanced_model' in globals() else None,
        hidden_size=128,
        dropout=0.2,
        lr=0.001,
        batch_size=32,
        epochs=1000,
        validation_ratio=0.15,
        patience=25,
        is_test=False
    )

    # Wizualizacja procesu uczenia
    plt.figure(figsize=(12, 6))
    plt.plot(fusion_history['train_loss'], label='Train Loss')
    if fusion_history['val_mae'] is not None:
        plt.plot(fusion_history['val_mae'], label='Val MAE')
    plt.title('Proces uczenia finalnego modelu uwagi (Attention Fusion)')
    plt.xlabel('Epoka')
    plt.ylabel('Wartość')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Wizualizacja wag uwagi
    if have_model_predictions:
        sample_indices = np.random.choice(len(y), size=5, replace=False)

        # Przygotowanie predykcji modeli
        nn_pred_viz = oof_predictions[sample_indices, 0].reshape(-1, 1)
        gnn_pred_viz = oof_predictions[sample_indices, 1].reshape(-1, 1) if oof_predictions.shape[1] > 1 else nn_pred_viz

        visualize_attention_weights(
            fusion_model,
            nn_pred_viz,
            gnn_pred_viz,
            X_orig_processed[sample_indices],
            sample_idx=None
        )

def cross_validate_attention_fusion(nn_predictions, gnn_predictions, orig_features, y,
                                   hidden_size=64, dropout=0.2, lr=0.001,
                                   n_splits=5, epochs=50, patience=10):
    """
    Przeprowadza cross-walidację dla modelu uwagi.

    Args:
        nn_predictions: Predykcje z sieci neuronowej
        gnn_predictions: Predykcje z GNN
        orig_features: Oryginalne cechy
        y: Wartości docelowe
        hidden_size: Rozmiar warstw ukrytych
        dropout: Współczynnik dropout
        lr: Learning rate
        n_splits: Liczba podziałów w cross-walidacji
        epochs: Maksymalna liczba epok
        patience: Liczba epok do early stopping

    Returns:
        tuple: (mean_mae, std_mae, all_maes)
    """
    print(f"Rozpoczęcie {n_splits}-krotnej cross-walidacji modelu uwagi...")

    # Przygotowanie KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Przygotowanie predykcji i cech
    nn_pred = nn_predictions.reshape(-1, 1) if nn_predictions.ndim == 1 else nn_predictions
    gnn_pred = gnn_predictions.reshape(-1, 1) if gnn_predictions.ndim == 1 else gnn_predictions

    # Listy do przechowywania wyników
    all_maes = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(y)):
        print(f"\nFold {fold+1}/{n_splits}")

        # Podział danych
        nn_train, nn_val = nn_pred[train_idx], nn_pred[val_idx]
        gnn_train, gnn_val = gnn_pred[train_idx], gnn_pred[val_idx]
        orig_train, orig_val = orig_features[train_idx], orig_features[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Trening modelu
        fusion_model, history, val_mae = train_final_attention_fusion_model(
            None,
            orig_train,
            y_train,
            hidden_size=hidden_size,
            dropout=dropout,
            lr=lr,
            batch_size=32,
            epochs=epochs,
            validation_ratio=0.0,
            patience=patience,
            is_test=True,
            test_predictions=np.hstack([nn_val, gnn_val]),
            test_features=orig_val,
            test_y=y_val
        )

        all_maes.append(val_mae)
        print(f"Fold {fold+1} MAE: {val_mae:.4f}")

    # Obliczenie statystyk
    mean_mae = np.mean(all_maes)
    std_mae = np.std(all_maes)

    print(f"\nWyniki cross-walidacji:")
    print(f"Średni MAE: {mean_mae:.4f} ± {std_mae:.4f}")
    print(f"Wszystkie MAE: {[f'{mae:.4f}' for mae in all_maes]}")

    return mean_mae, std_mae, all_maes

if is_test_iteration:
    print("\nPrzeprowadzanie cross-walidacji dla modelu uwagi...")

    have_predictions = False
    if 'oof_predictions' in globals() and oof_predictions is not None:
        have_predictions = True
        nn_cross_preds = oof_predictions[:, 0]
        gnn_cross_preds = oof_predictions[:, 1] if oof_predictions.shape[1] > 1 else nn_cross_preds
        print(f"Używanie istniejących predykcji z {oof_predictions.shape[1]} modeli bazowych")
    else:
        print("Nie znaleziono predykcji modeli bazowych - wychodzenie...")

    if have_predictions:
        # cross-walidacja
        mean_mae, std_mae, all_maes = cross_validate_attention_fusion(
            nn_cross_preds,
            gnn_cross_preds,
            X_orig_processed,
            y,
            hidden_size=128,
            dropout=0.2,
            lr=0.001,
            n_splits=5,
            epochs=50,
            patience=10
        )

        # Porównanie z wcześniejszymi wynikami
        cv_results = {}

        if 'model_mae_scores' in globals() and 'NN' in model_mae_scores:
            cv_results['NN Model (CV)'] = model_mae_scores['NN']

        if 'final_enhanced_mae' in globals():
            cv_results['GNN Model (CV)'] = final_enhanced_mae

        # Dodanie wyniku modelu uwagi
        cv_results['Attention Fusion (CV)'] = mean_mae

        # Wizualizacja porównania
        if cv_results:
            plt.figure(figsize=(10, 6))
            colors = ['skyblue', 'green', 'red']
            plt.bar(
                range(len(cv_results)),
                list(cv_results.values()),
                color=colors[:len(cv_results)]
            )
            plt.xticks(range(len(cv_results)), list(cv_results.keys()), rotation=45)
            plt.title('Porównanie MAE w cross-walidacji')
            plt.ylabel('MAE')
            plt.grid(True, linestyle='--', alpha=0.3, axis='y')
            plt.tight_layout()
            plt.show()




In [ ]:
# Predykcje
def space_formatter(x, pos):
    return f"{int(x):,}".replace(',', ' ')

predictions_combined = []

if is_test_iteration:
    print("Wykonywanie predykcji z wykorzystaniem pełnego modelu kaskadowego z mechanizmem uwagi...")

    # WARSTWA 1: Predykcje bazowych modeli
    print("Warstwa 1: Predykcje modeli bazowych...")
    preds_xgb = xgb_model.predict(X_valid_end)
    preds_cat = cat_model.predict(X_valid_end)
    preds_lgb = lgb_model.predict(X_valid_end)
    preds_ada = ada_pipeline.predict(X_valid_end)
    preds_hgbm = hgbm_model.predict(X_valid_end)

    print(f"XGB MAE: {mean_absolute_error(y_valid_end, preds_xgb):.4f}")
    print(f"CatBoost MAE: {mean_absolute_error(y_valid_end, preds_cat):.4f}")
    print(f"LGBM MAE: {mean_absolute_error(y_valid_end, preds_lgb):.4f}")
    print(f"AdaBoost MAE: {mean_absolute_error(y_valid_end, preds_ada):.4f}")
    print(f"HGBM MAE: {mean_absolute_error(y_valid_end, preds_hgbm):.4f}")

    layer1_features = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))

    layer1_features_scaled = scaler.transform(layer1_features)

    # WARSTWA 2: Predykcje modeli drugiej warstwy
    print("Warstwa 2: Predykcje z modeli pośrednich...")
    layer2_predictions = {}

    for model_name, model in models_dict.items():
        if model_name == "NN":
            preds = model.predict(layer1_features_scaled, verbose=0).flatten()
        elif model_name == "MLP":
            preds = model.predict(layer1_features_scaled)
        else:
            preds = model.predict(layer1_features)

        layer2_predictions[model_name] = preds
        print(f"  - Model {model_name}: MAE = {mean_absolute_error(y_valid_end, layer2_predictions[model_name]):.4f}")

    layer2_features = np.column_stack(list(layer2_predictions.values()))

    # WARSTWA 3: Trenowanie finalnych modeli meta z wykorzystaniem predykcji z warstwy 2
    print("Warstwa 3: Generowanie predykcji z finalnych modeli...")

    nn_test_preds = predict_with_nn_model(final_meta_model, scaler_final.transform(layer2_features)).reshape(-1, 1)
    print(f"  - Model NN (sieć neuronowa): MAE = {mean_absolute_error(y_valid_end, nn_test_preds):.4f}")

    X_valid_orig_processed = orig_preprocessor.transform(X_valid_end)

    predictions = predict_with_enhanced_gnn(
        final_enhanced_model,
        layer2_features,
        X_valid_orig_processed,
        scaler_pred
    ).reshape(-1, 1)
    print(f"  - Model GNN (grafowa sieć neuronowa): MAE = {mean_absolute_error(y_valid_end, predictions):.4f}")

    predictions = predictions.flatten()

    # Porównanie wyników wszystkich głównych modeli
    comparison_results = {
        "NN (sieć neuronowa)": mean_absolute_error(y_valid_end, nn_test_preds),
        "GNN (grafowa sieć)": mean_absolute_error(y_valid_end, predictions),
    }

    # Dodaj najlepsze modele z warstwy 2
    best_layer2_models = sorted([(name, mean_absolute_error(y_valid_end, preds))
                                for name, preds in layer2_predictions.items()],
                               key=lambda x: x[1])[:3]

    for name, mae in best_layer2_models:
        comparison_results[f"Warstwa 2: {name}"] = mae

    # Wizualizacja porównania wyników
    plt.figure(figsize=(12, 6))
    names = list(comparison_results.keys())
    values = list(comparison_results.values())
    colors = ['skyblue', 'green', 'red'] + ['gray'] * len(best_layer2_models)

    plt.bar(range(len(comparison_results)), values, color=colors)
    plt.xticks(range(len(comparison_results)), names, rotation=45, ha='right')
    plt.title('Porównanie MAE różnych modeli na zbiorze testowym')
    plt.ylabel('MAE (niższe = lepsze)')
    plt.grid(True, linestyle='--', alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

    # Losowy wybór przypadków
    random_indices = np.random.choice(len(y_valid_end), min(len(y_valid_end), len(y_valid_end)), replace=False)

    # Tabela z wartościami
    result_table = pd.DataFrame({
        'Indeks': random_indices,
        'NN Predykcja': nn_test_preds[random_indices, 0],
        'GNN Predykcja': gnn_test_preds[random_indices, 0],
        'Finalna Predykcja': predictions[random_indices],
        'Wartość Rzeczywista': y_valid_end.iloc[random_indices].values if isinstance(y_valid_end, pd.Series) else
        y_valid_end[random_indices]
    })

    display(result_table.round(4))

    plt.figure(figsize=(20, 10))
    plt.hist(predictions, bins=20, alpha=0.7, color='blue', label='Predykcje')
    plt.xlabel("Wartość błędu")
    plt.ylabel("Liczebność")
    plt.title("Histogram predykcji")
    plt.ticklabel_format(style='plain', axis='x')

    ax = plt.gca()
    ax.xaxis.set_major_formatter(FuncFormatter(space_formatter))

    plt.legend()
    plt.show()

    # Wykres porównawczy predykcji vs wartości rzeczywiste
    plt.figure(figsize=(12, 8))
    plt.scatter(y_valid_end, predictions, alpha=0.5)
    plt.plot([y_valid_end.min(), y_valid_end.max()], [y_valid_end.min(), y_valid_end.max()], 'r--')
    plt.xlabel('Wartości rzeczywiste')
    plt.ylabel('Predykcje')
    plt.title('Porównanie predykcji z wartościami rzeczywistymi')
    plt.ticklabel_format(style='plain')

    # Formatowanie osi
    ax = plt.gca()
    ax.xaxis.set_major_formatter(FuncFormatter(space_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(space_formatter))

    plt.tight_layout()
    plt.show()



In [ ]:
if is_test_iteration:
    x = np.arange(len(y_valid_end))
    y_valid_array = y_valid_end.to_numpy().flatten()
    predictions_array = np.array(predictions).flatten()

    plt.figure(figsize=(10, 6))
    # Wykres punktowy dla wartości rzeczywistych (niebieski)
    plt.scatter(x, y_valid_end, color='blue', label='Wartość rzeczywista')

    # Wykres punktowy dla wartości estymowanych (czerwony)
    plt.scatter(x, predictions, color='red', label='Wartość estymowana')

    # Dla każdej pary linia łącząca punkty
    for i in range(len(x)):
        plt.plot([x[i], x[i]], [y_valid_array[i], predictions_array[i]], color='gray', linewidth=0.5)

    plt.xlabel('Indeks')
    plt.ylabel('Wartość')
    plt.title(f'Porównanie wartości rzeczywistych i estymowanych')
    plt.legend()

    ax = plt.gca()
    ax.yaxis.set_major_formatter(FuncFormatter(space_formatter))

    plt.show()


In [ ]:
if not is_test_iteration:
    valid_preds = pd.read_csv('data/Test.csv')

    corrections = {
        'SWIZERLAND': 'SWITZERLAND',
        'UNITED STATES OF AMERICA': 'UNITED STATES',
        'COMORO': 'COMOROS',
        'MALT': 'MALTA',
        'UAE': 'UNITED ARAB EMIRATES',
        'UKRAIN': 'UKRAINE',
        'DRC': 'CONGO (DEMOCRATIC REPUBLIC OF THE)',
        'SWAZILAND': 'ESWATINI',
        'COSTARICA': 'COSTA RICA',
        'SCOTLAND': 'UNITED KINGDOM',
        'PHILIPINES': 'PHILIPPINES',
        'BOSNIA': 'BOSNIA AND HERZEGOVINA',
        'CAPE VERDE': 'CABO VERDE',
        'MORROCO': 'MOROCCO',
        'SOMALI': 'SOMALIA',
        'KOREA': 'SOUTH KOREA',
        'SAUD ARABIA': 'SAUDI ARABIA',
    }

    valid_preds['country'] = valid_preds['country'].replace(corrections)

    valid_preds = pd.merge(valid_preds, df_regions, how='left', left_on='country', right_on='name')
    valid_preds = valid_preds.drop(columns=['name'])
    print(valid_preds.columns)
    print(df_regions.columns)

    # Check before cleansing
    for column in valid_preds.columns:
        empty_count = valid_preds[column].isna().sum()

        if empty_count > 0:
            print(f"Column '{column}' has {empty_count} empty fields (NaN).")

    valid_preds.loc[valid_preds['most_impressing'].isna(), 'most_impressing'] = 'No comments'

    # Travel with imputation
    features_tw = ['region', 'age_group', 'total_female',
           'total_male', 'purpose', 'main_activity', 'info_source',
           'tour_arrangement', 'package_transport_int', 'package_accomodation',
           'package_food', 'package_transport_tz', 'package_sightseeing',
           'package_guided_tour', 'package_insurance', 'night_mainland',
           'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing']

    features_cat_tw = ['region', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                    'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                    'package_sightseeing', 'package_guided_tour', 'package_insurance',
                    'payment_mode', 'first_trip_tz', 'most_impressing']


    # Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
    valid_preds_no_nan = valid_preds.dropna()

    allowed_categories = ['Children', 'Friends/Relatives', 'Spouse', 'Spouse and Children']
    valid_preds_imp = valid_preds_no_nan[valid_preds_no_nan['travel_with'].isin(allowed_categories)].copy()
    X_train_tw = pd.get_dummies(valid_preds_imp[features_tw], columns=features_cat_tw)
    y_train_tw = valid_preds_imp['travel_with']

    # Trenowanie modelu
    rf_tw = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_tw.fit(X_train_tw, y_train_tw)

    print("Cleansing test dataset")
    for index, row in valid_preds[valid_preds.isna().any(axis=1)].iterrows():
        if pd.isna(row['travel_with']) & (row['total_male'] + row['total_female'] == 1) & (not pd.isna(row['total_male'])) & (not pd.isna(row['total_female'])):
            valid_preds.loc[index, 'travel_with'] = 'Alone'
        if pd.isna(row['travel_with']) & (row['total_male'] + row['total_female'] > 1) & (not pd.isna(row['total_male'])) & (not pd.isna(row['total_female'])):
            dummy_df = pd.get_dummies(row[features_tw], columns=features_cat_tw)
            dummy_df.columns = dummy_df.columns.astype(str)
            dummy_df = dummy_df.reindex(columns=X_train_tw.columns, fill_value=0)

            predicted_value = rf_tw.predict(dummy_df)

            print(f"\nrow"
                  f"\n{row['travel_with']}"
                  f"\n{row['total_male']}"
                  f"\n{row['total_female']}"
                  f"\n{predicted_value[0]}")

            valid_preds.loc[index, 'travel_with'] = predicted_value[0]

        if (row['travel_with'] == 'Alone') & ((pd.isna(row['total_male'])) | (pd.isna(row['total_female']))):
            if pd.isna(row['total_female']):
                valid_preds.loc[index, 'total_female'] = 0

                print(f"\nrow"
                      f"\n{row['travel_with']}"
                      f"\n{row['total_male']}"
                      f"\n{row['total_female']}"
                      f"\n{0}")
            elif pd.isna(row['total_female']):
                valid_preds.loc[index, 'total_male'] = 0

                print(f"\nrow"
                      f"\n{row['travel_with']}"
                      f"\n{row['total_male']}"
                      f"\n{row['total_female']}"
                      f"\n{0}")
            elif pd.isna(row['total_male']) & pd.isna(row['total_female']):
                # Gentleman
                valid_preds.loc[index, 'total_male'] = 0
                valid_preds.loc[index, 'total_female'] = 1

        elif (pd.isna(row['total_female'])) & (row['total_male'] > 0):

            if row['total_male'] > 1:
                valid_preds.loc[index, 'total_female'] = 0
            else:
                valid_preds.loc[index, 'total_female'] = 1

            print(f"\nrow"
                  f"\n{row['travel_with']}"
                  f"\n{row['total_male']}"
                  f"\n{row['total_female']}"
                  f"\n{0}")

        elif (pd.isna(row['total_male'])) & (row['total_female'] > 0):

            if row['total_female'] > 1:
                valid_preds.loc[index, 'total_male'] = 0
            else:
                valid_preds.loc[index, 'total_male'] = 1

            print(f"\nrow"
                  f"\n{row['travel_with']}"
                  f"\n{row['total_male']}"
                  f"\n{row['total_female']}"
                  f"\n{0}")

        if (row['total_male'] + row['total_female'] == 0) & (not pd.isna(row['total_male'])) & (not pd.isna(row['total_female'])):

            # Gentleman
            valid_preds.loc[index, 'total_female'] = 1

            if pd.isna(row['travel_with']):
                dummy_df = pd.get_dummies(row[features_tw], columns=features_cat_tw)
                dummy_df.columns = dummy_df.columns.astype(str)
                dummy_df = dummy_df.reindex(columns=X_train_tw.columns, fill_value=0)

                predicted_value = rf_tw.predict(dummy_df)

                valid_preds.loc[index, 'travel_with'] = predicted_value[0]

            print(f"\nrow widmo"
                  f"\n{row['travel_with']}"
                  f"\n{row['total_male']}"
                  f"\n{row['total_female']}")

    # Check after cleansing
    for index, row in valid_preds[valid_preds.isna().any(axis=1)].iterrows():
        if pd.isna(row['travel_with']) | pd.isna(row['total_male']) | pd.isna(row['total_female']):
            print(f"\nrow"
                  f"\n{row['travel_with']}"
                  f"\n{row['total_male']}"
                  f"\n{row['total_female']}")

    valid_preds['total_people'] = valid_preds['total_male'] + valid_preds['total_female']
    valid_preds['night_total'] = valid_preds['night_zanzibar'] + valid_preds['night_mainland']

    for column in valid_preds.columns:
        empty_count = valid_preds[column].isna().sum()

        if empty_count > 0:
            print(f"Column '{column}' has {empty_count} empty fields (NaN).")


    unique_countries_nan = valid_preds[valid_preds['region'].isna()]['country'].unique()
    print("\nPaństwa bez regionu: ", unique_countries_nan)

    for col in valid_preds.select_dtypes(include=['object']).columns:
        if col != "ID":
            valid_preds[col] = valid_preds[col].astype('category')
            new_col = "cat_" + str(col)
            valid_preds.rename(columns={col: new_col}, inplace=True)
    print(valid_preds.columns)
    valid_preds.drop(columns=['cat_most_impressing', 'cat_country'], inplace=True)
    print(valid_preds.columns)


Challange predictions

In [ ]:
if not is_test_iteration:
    id_valid = valid_preds['ID']
    X_valid_preds = valid_preds.drop('ID', axis=1)

    # WARSTWA 1: Predykcje bazowych modeli
    print("Warstwa 1: Predykcje modeli bazowych...")
    preds_xgb = xgb_model.predict(X_valid_preds)
    preds_cat = cat_model.predict(X_valid_preds)
    preds_lgb = lgb_model.predict(X_valid_preds)
    preds_ada = ada_pipeline.predict(X_valid_preds)
    preds_hgbm = hgbm_model.predict(X_valid_preds)

    layer1_features = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))
    layer1_features_scaled = scaler.transform(layer1_features)

    # WARSTWA 2: Predykcje modeli drugiej warstwy
    print("Warstwa 2: Predykcje z modeli pośrednich...")
    layer2_predictions = {}

    for model_name, model in models_dict.items():
        if model_name == "NN":
            preds = model.predict(layer1_features_scaled, verbose=0).flatten()
        elif model_name == "MLP":
            preds = model.predict(layer1_features_scaled)
        else:
            preds = model.predict(layer1_features)

        layer2_predictions[model_name] = preds

    layer2_features = np.column_stack(list(layer2_predictions.values()))


    # WARSTWA 3: Generowanie predykcji z finalnych modeli
    print("Warstwa 3: Generowanie predykcji z finalnych modeli...")

    # 1. Predykcje z modelu NN
    nn_final_preds = predict_with_nn_model(final_meta_model, scaler_final.transform(layer2_features)).reshape(-1, 1)
    nn_final_preds = nn_final_preds.flatten()

    # Tworzenie i zapis pliku submission_nn.csv
    submission_nn = pd.DataFrame({
        'ID': id_valid,
        'total_cost': nn_final_preds
    })
    submission_nn.to_csv('data/submission_nn.csv', index=False)
    print(f"Zapisano predykcje modelu NN do pliku 'data/submission_nn.csv'")

    # 2. Predykcje z modelu GNN
    # Przetworzenie oryginalnych cech dla GNN
    X_valid_orig_processed = orig_preprocessor.transform(X_valid_preds)

    gnn_final_preds = predict_with_enhanced_gnn(
        final_enhanced_model,
        layer2_features,
        X_valid_orig_processed,
        scaler_pred
    ).reshape(-1, 1)
    gnn_final_preds = gnn_final_preds.flatten()

    # Tworzenie i zapis pliku submission_gnn.csv
    submission_gnn = pd.DataFrame({
        'ID': id_valid,
        'total_cost': gnn_final_preds
    })
    submission_gnn.to_csv('data/submission_gnn.csv', index=False)
    print(f"Zapisano predykcje modelu GNN do pliku 'data/submission_gnn.csv'")

    # Statystyki predykcji
    print("\nStatystyki predykcji:")
    print(f"NN - min: {nn_final_preds.min():.2f}, max: {nn_final_preds.max():.2f}, mean: {nn_final_preds.mean():.2f}")
    print(f"GNN - min: {gnn_final_preds.min():.2f}, max: {gnn_final_preds.max():.2f}, mean: {gnn_final_preds.mean():.2f}")

    # testy: średnia z obu modeli jako ensemble
    ensemble_preds = (nn_final_preds + gnn_final_preds) / 2
    submission_ensemble = pd.DataFrame({
        'ID': id_valid,
        'total_cost': ensemble_preds
    })
    submission_ensemble.to_csv('data/submission_ensemble.csv', index=False)
    print(f"Zapisano predykcje ensembla (średnia NN+GNN) do pliku 'data/submission_ensemble.csv'")


    # Łączenie predykcji dla meta modelu
    # meta_features_new = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))

    # Przeskalowanie
    # meta_features_new_scaled = scaler.transform(meta_features_new)

    # Predykcja finalna
    # predictions = meta_model.predict(meta_features_new_scaled)

    # if predictions.ndim > 1 and predictions.shape[1] == 1:
    #     predictions = predictions.ravel()
    #
    # # Finalne dane predykcji
    # results = pd.DataFrame({
    #     'ID': id_valid,
    #     'total_cost': predictions
    # })
    #
    #
    # results.to_csv('data/submission.csv', index=False)


Sieć neuronowa:

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, LSTM, Flatten

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
        "MAPE": mape(y_true, y_pred)
    }


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["ID", "total_cost"])
y = train["total_cost"]
X_test_final = test.drop(columns=["ID"])

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test_final)

X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, name):
    start_time = time.time()
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    end_time = time.time()
    y_pred = model.predict(X_val).flatten()
    metrics = evaluate_metrics(y_val, y_pred)
    metrics['Model'] = name
    metrics['TrainTime'] = end_time - start_time
    return metrics, y_pred


In [ ]:
X_train_seq = np.expand_dims(X_train.toarray() if hasattr(X_train, 'toarray') else X_train, axis=-1)
X_val_seq = np.expand_dims(X_val.toarray() if hasattr(X_val, 'toarray') else X_val, axis=-1)
input_shape_seq = X_train_seq.shape[1:]
input_shape_flat = X_train.shape[1]

In [ ]:
def build_mlp():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape_flat,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_cnn():
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=input_shape_seq),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_lstm():
    model = Sequential([
        LSTM(64, input_shape=input_shape_seq),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_resnet():
    inputs = tf.keras.Input(shape=(input_shape_flat,))
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    residual = x
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.add([x, residual])
    outputs = Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

def build_autoencoder():
    input_layer = tf.keras.Input(shape=(input_shape_flat,))
    encoded = Dense(64, activation='relu')(input_layer)
    decoded = Dense(input_shape_flat, activation='linear')(encoded)
    autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [ ]:
results = []
model_builders = [
    (build_mlp, "MLP"),
    (build_cnn, "CNN"),
    (build_lstm, "LSTM"),
    (build_resnet, "ResNetMLP")
]

for builder, name in model_builders:
    model = builder()
    X_tr = X_train_seq if "CNN" in name or "LSTM" in name else X_train
    X_vl = X_val_seq if "CNN" in name or "LSTM" in name else X_val
    metrics, _ = train_model(model, X_tr, y_train, X_vl, y_val, name)
    results.append(metrics)

autoencoder = build_autoencoder()
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=0)
X_train_encoded = autoencoder.predict(X_train)
X_val_encoded = autoencoder.predict(X_val)
mlp_on_ae = build_mlp()
metrics, _ = train_model(mlp_on_ae, X_train_encoded, y_train, X_val_encoded, y_val, "Autoencoder+MLP")
results.append(metrics)


In [ ]:
esults_df = pd.DataFrame(results)
print(results_df)

plt.figure(figsize=(12, 6))
sns.barplot(data=results_df, x="Model", y="R2")
plt.title("R² Scores of Models")
plt.show()

results_df.to_csv("model_results.csv", index=False)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, LSTM, Flatten, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.ensemble import AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

class BaseModel:
    def fit(self, X, y, **kwargs):
        return self.model.fit(X, y, **kwargs)

    def predict(self, X):
        return self.model.predict(X)

    def evaluate(self, X, y):
        return self.model.evaluate(X, y)


class SimpleMLP(BaseModel):
    def __init__(self, input_shape, output_dim=1):
        self.model = Sequential([
            Dense(128, activation='relu', input_shape=(input_shape,)),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dense(output_dim)
        ])
        self.model.compile(optimizer='adam', loss='mse')


class ConvNet1D(BaseModel):
    def __init__(self, input_shape, output_dim=1):
        self.model = Sequential([
            Conv1D(64, 3, activation='relu', input_shape=input_shape),
            MaxPooling1D(2),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(output_dim)
        ])
        self.model.compile(optimizer='adam', loss='mse')


class LSTMRegressor(BaseModel):
    def __init__(self, input_shape, output_dim=1):
        self.model = Sequential([
            LSTM(64, input_shape=input_shape),
            Dense(output_dim)
        ])
        self.model.compile(optimizer='adam', loss='mse')


class ResNetMLP(BaseModel):
    def __init__(self, input_shape, output_dim=1):
        inputs = tf.keras.Input(shape=(input_shape,))
        x = Dense(128, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        residual = x
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = tf.keras.layers.add([x, residual])
        outputs = Dense(output_dim)(x)
        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        self.model.compile(optimizer='adam', loss='mse')


class ConfigurableNeuralNetwork(BaseModel):
    def __init__(self, input_shape, network_type='mlp', output_dim=1,
                 hidden_units=[128, 64], dropout_rate=0.2, optimizer='adam'):
        self.input_shape = input_shape
        self.network_type = network_type
        self.output_dim = output_dim
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
        self.optimizer = optimizer
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        if self.network_type == 'mlp':
            model.add(Dense(self.hidden_units[0], activation='relu', input_shape=(self.input_shape,)))
            model.add(Dropout(self.dropout_rate))
            for units in self.hidden_units[1:]:
                model.add(Dense(units, activation='relu'))
                model.add(Dropout(self.dropout_rate))
        elif self.network_type == 'cnn':
            model.add(Conv1D(64, 3, activation='relu', input_shape=self.input_shape))
            model.add(MaxPooling1D(2))
            model.add(Flatten())
        elif self.network_type == 'lstm':
            model.add(LSTM(64, input_shape=self.input_shape))
        elif self.network_type == 'cnn_lstm':
            model.add(Conv1D(64, 3, activation='relu', input_shape=self.input_shape))
            model.add(MaxPooling1D(2))
            model.add(LSTM(64))
        model.add(Dense(self.output_dim))
        model.compile(optimizer=self.optimizer, loss='mse')
        return model

    def fit(self, X, y, **kwargs):
        callbacks = [
            EarlyStopping(patience=5, restore_best_weights=True),
            ReduceLROnPlateau(patience=3)
        ]
        return self.model.fit(X, y, callbacks=callbacks, **kwargs)

    @classmethod
    def load_model(cls, filepath, **kwargs):
        model = tf.keras.models.load_model(filepath)
        input_shape = model.input_shape[1:]
        output_dim = model.output_shape[-1]
        instance = cls(input_shape=input_shape, output_dim=output_dim, **kwargs)
        instance.model = model
        return instance


xgb_model = XGBRegressor()
cat_model = CatBoostRegressor(verbose=0)
lgb_model = LGBMRegressor()
ada_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', AdaBoostRegressor())
])
hgbm_model = HistGradientBoostingRegressor()

ridge_stack = Ridge()

final_stack = LinearRegression()


# xgb_model.fit(X_train_preds, y_train_preds)
# preds_xgb = xgb_model.predict(X_valid_preds)

# layer1_features = np.column_stack((preds_xgb, preds_cat, preds_lgb, preds_ada, preds_hgbm))
# ridge_stack.fit(layer1_features, y_valid_preds)
# preds_ridge = ridge_stack.predict(layer1_features)

# final_stack.fit(np.column_stack((preds_ridge,)), y_valid_preds)
# preds_final = final_stack.predict(np.column_stack((preds_ridge,)))